# Generate miners.json file and performs initial 3rd party attribution for blocks_attribition.json 

This notebook generates:
* `miners.json` miners files containing mining pool addresses and markers. 
* `blocks_attribution_0-$(current_blockheight).json` attributes miners to blocks based on third party information

### Types of attributions:
The added attributions are **only** based on the respective sources. Every updated to the *miners* made during the attribution are only relevant for future (custom) attributions:
- `blockchain_info_tag`: (unique) from pools.json available on github from blockchain.info, use the pool tag. This information is serves in miners_initial.json
- `blockchain_info_address`: (one for each output address) from pools.json available on github from blockchain.info, use the output addresses info 
- `blockchain_info`: which is the union of the above attributions
- `blocktrail`: (unique) use data from blocktrail.com 
- `btcccom_*`: also attribution form this file which has same syntax as blockchain.info file

## Imports

In [22]:
# python3.5
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import re
import binascii
import sys
import random # draw samples

import csv
import json
from collections import Counter
from collections import defaultdict
import copy

import os.path # os.path.isfile; 

# dict diffing and printing
import dictdiffer
import pprint
pp = pprint.PrettyPrinter(indent=4)

In [23]:
# custom imports 
import util
from importlib import reload
reload(util)

<module 'util' from '/home/matteo/deep_dive/util.py'>

## Global variables and functions

In [24]:
# data up to blockheight 
current_blockheight = util.CURRENT_BLOCKHEIGHT
print(current_blockheight)

556400


### Input files

#### `blocks_0-$(current_blockheight).json`
The block.sjon file filled with raw data form the blockchain

In [25]:
blocks_json_file = "./dataset/blocks_0-" + str(current_blockheight) + ".json"
assert( os.path.isfile(blocks_json_file) )

#### `blockchain.info_$(date -I).json`
pools file from blockchain.info from github
* https://github.com/blockchain/Blockchain-Known-Pools
* https://github.com/blockchain/Blockchain-Known-Pools/tree/82ed31956388e3950845cc2faeaf6679a057ee5b
* UPDATE 2019-01-28: https://github.com/blockchain/Blockchain-Known-Pools/tree/29ab27c844ebdb63110f8783f73b9decd4abc221

In [26]:
!pwd

/home/matteo/deep_dive


In [27]:
!wget --output-document='./dataset/blockchain.info_2019-01-28.json' "https://raw.githubusercontent.com/blockchain/Blockchain-Known-Pools/29ab27c844ebdb63110f8783f73b9decd4abc221/pools.json"

--2019-06-03 06:37:07--  https://raw.githubusercontent.com/blockchain/Blockchain-Known-Pools/29ab27c844ebdb63110f8783f73b9decd4abc221/pools.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.192.133, 151.101.128.133, 151.101.64.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.192.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 22210 (22K) [text/plain]
Saving to: ‘./dataset/blockchain.info_2019-01-28.json’

./dataset/blockchai 100%[===================>]  21,69K  --.-KB/s    in 0,003s  

2019-06-03 06:37:07 (6,48 MB/s) - ‘./dataset/blockchain.info_2019-01-28.json’ saved [22210/22210]



In [28]:
#pools_blockchain_info_json_file = './miners_and_pools_sources/blockchain.info_2018-03-10.json'
pools_blockchain_info_json_file = './dataset/blockchain.info_2019-01-28.json'
assert( os.path.isfile(pools_blockchain_info_json_file) )

#### `btccom_$(date -I).json`
pools file from btccom from github
* https://github.com/btccom/Blockchain-Known-Pools
* https://raw.githubusercontent.com/btccom/Blockchain-Known-Pools/650a92227bf65b06ff0a5b58bb57c13856a3babf/pools.json

In [29]:
!pwd

/home/matteo/deep_dive


In [30]:
!wget --output-document='./dataset/btccom_2019-02-15.json' "https://raw.githubusercontent.com/btccom/Blockchain-Known-Pools/650a92227bf65b06ff0a5b58bb57c13856a3babf/pools.json"

--2019-06-03 06:37:07--  https://raw.githubusercontent.com/btccom/Blockchain-Known-Pools/650a92227bf65b06ff0a5b58bb57c13856a3babf/pools.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.192.133, 151.101.128.133, 151.101.64.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.192.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 25075 (24K) [text/plain]
Saving to: ‘./dataset/btccom_2019-02-15.json’

./dataset/btccom_20 100%[===================>]  24,49K  --.-KB/s    in 0,004s  

2019-06-03 06:37:07 (6,54 MB/s) - ‘./dataset/btccom_2019-02-15.json’ saved [25075/25075]



In [31]:
#pools_blockchain_info_json_file = './miners_and_pools_sources/blockchain.info_2018-03-10.json'
pools_btccom_json_file = './dataset/btccom_2019-02-15.json'
assert( os.path.isfile(pools_btccom_json_file) )

#### `blocks_blocktrail_0-514240.json`
Attribution information fetched from blocktrail.com 
This file was generated with `blocktrail_extract.py` from previously fetched info since API has changed

new api:
* https://www.blocktrail.com/api

In [32]:
blocks_blocktrail_json_file = "./dataset/blocks_blocktrail_0-514240.json"
assert( os.path.isfile(blocks_blocktrail_json_file) )

### Output files

#### `miners_initial_blockchaininfo.json`
The mining entity data from:
* blockchain.info 

in our custom JSON format defined above

In [33]:
miners_initial_blockchaininfo_json_file = './dataset/miners_initial_blockchaininfo.json'
if os.path.isfile(miners_initial_blockchaininfo_json_file):
    print("Output file " + miners_initial_blockchaininfo_json_file +  "exists, will be overwritten.")

Output file ./dataset/miners_initial_blockchaininfo.jsonexists, will be overwritten.


#### `miners_initial_blockchaininfo_conflicts.json`
The mining entity data from:
* blockchain.info 

in our custom JSON format defined above

In [34]:
miners_initial_blockchaininfo_conflicts_json_file = './dataset/miners_initial_blockchaininfo_conflicts.json'
if os.path.isfile(miners_initial_blockchaininfo_conflicts_json_file):
    print("Output file " + miners_initial_blockchaininfo_conflicts_json_file +  "exists, will be overwritten.")

Output file ./dataset/miners_initial_blockchaininfo_conflicts.jsonexists, will be overwritten.


#### `miners_initial_btccom.json`
The mining entity data from:
* btccom 

in our custom JSON format defined above

In [35]:
miners_initial_btccom_json_file = './dataset/miners_initial_btccom.json'
if os.path.isfile(miners_initial_btccom_json_file):
    print("Output file " + miners_initial_btccom_json_file +  "exists, will be overwritten.")

Output file ./dataset/miners_initial_btccom.jsonexists, will be overwritten.


#### `miners_initial_btccom_conflicts.json`
The mining entity data from:
* btccom 

in our custom JSON format defined above

In [36]:
miners_initial_btccom_conflicts_json_file = './dataset/miners_initial_btccom_conflicts.json'
if os.path.isfile(miners_initial_btccom_conflicts_json_file):
    print("Output file " + miners_initial_btccom_conflicts_json_file +  "exists, will be overwritten.")

Output file ./dataset/miners_initial_btccom_conflicts.jsonexists, will be overwritten.


#### `miners_initial_incl_blocktrail.json`
The mining entity data from:
* incl some miners identified by blocktrail 

in our custom JSON format defined above

In [37]:
miners_initial_incl_blocktrail_json_file = './dataset/miners_initial_incl_blocktrail.json'
if os.path.isfile(miners_initial_incl_blocktrail_json_file):
    print("Output file " + miners_initial_incl_blocktrail_json_file +  "exists, will be overwritten.")

Output file ./dataset/miners_initial_incl_blocktrail.jsonexists, will be overwritten.


#### `miners_initial.json`
The mining entity data from:
* blockchain.info
* btccom 

in our custom JSON format defined above

In [38]:
miners_initial_json_file = './dataset/miners_initial.json'
if os.path.isfile(miners_initial_json_file):
    print("Output file " + miners_initial_json_file +  "exists, will be overwritten.")

Output file ./dataset/miners_initial.jsonexists, will be overwritten.


#### `miners_initial_conflicts.json`
The mining entity data from:
* blockchain.info
* btccom 

in our custom JSON format defined above

In [39]:
miners_initial_conflicts_json_file = './dataset/miners_initial_conflicts.json'
if os.path.isfile(miners_initial_conflicts_json_file):
    print("Output file " + miners_initial_conflicts_json_file +  "exists, will be overwritten.")

Output file ./dataset/miners_initial_conflicts.jsonexists, will be overwritten.


#### `miners.json`
The mining entity data from:
* blockchain.info 
* blocktrail.com

Again this data is in our custom JSON format defined above.

In [40]:
miners_json_file = './dataset/miners.json'
if os.path.isfile(miners_json_file):
    print("Output file " + miners_json_file +  "exists, will be overwritten.")

Output file ./dataset/miners.jsonexists, will be overwritten.


#### `blocks_attribution_0-$(current_blockheight).json`
already performed block attributions to link same miners and pools together:
- `blockchain_info_marker`
- `blockchain_info_address`
- `blockchain_info`
- `blocktrail`
- `btccom`
- `btccom_address`
- `btccom_marker`

In [41]:
blocks_attribution_json_file = './dataset/blocks_attribution_0-' + str(current_blockheight) + '.json'
if os.path.isfile(blocks_attribution_json_file):
    print("Output file " + blocks_attribution_json_file +  "exists, will be overwritten.")

Output file ./dataset/blocks_attribution_0-556400.jsonexists, will be overwritten.


## miners.json example structure which should get populated in this notebook


## Add pools file form blockcahin.info 

In [42]:
with open (pools_blockchain_info_json_file) as pools_bc_data:
    pools_bc = json.load(pools_bc_data)

In [43]:
for field in pools_bc:
    print(field)

coinbase_tags
payout_addresses


In [44]:
assert "payout_addresses" in pools_bc.keys()
assert "coinbase_tags" in pools_bc.keys()

In [45]:
len(pools_bc["payout_addresses"]) # 85

85

In [46]:
len(pools_bc["coinbase_tags"]) # 89

89

## Add pools file form btccom

Compare and add the info from the btccom file

In [47]:
with open (pools_btccom_json_file) as pools_btccom_data:
    pools_btccom = json.load(pools_btccom_data)

In [48]:
for field in pools_btccom:
    print(field)

coinbase_tags
payout_addresses


In [49]:
assert "payout_addresses" in pools_bc.keys()
assert "coinbase_tags" in pools_bc.keys()

In [50]:
len(pools_btccom["payout_addresses"]) # 93

93

In [51]:
len(pools_btccom["coinbase_tags"]) # 104

104

In [52]:
diffs = list(dictdiffer.diff(pools_bc,pools_btccom))
for diff in diffs:
    if diff[0] == "add":
        pprint.pprint(diff)
    if diff[0] == "remove":
        pprint.pprint(diff)

('add',
 'coinbase_tags',
 [('BITFARMS', {'link': 'https://www.bitarms.io/', 'name': 'Bitfarms'}),
  ('/Huobi/', {'link': 'https://www.poolhb.com/', 'name': 'Huobi.pool'}),
  ('/E2M & BTC.TOP/', {'link': 'http://www.easy2mine.com/', 'name': 'WAYI.CN'}),
  ('/canoepool/', {'link': 'https://www.canoepool.com/', 'name': 'CanoePool'}),
  ('Mined By AntPool', {'link': 'https://www.antpool.com/', 'name': 'AntPool'}),
  ('BWPool', {'link': 'https://bwpool.net/', 'name': 'BWPool'}),
  ('/DCEX/', {'link': 'http://dcexploration.cn', 'name': 'DCEX'}),
  ('/BTPOOL/', {'link': '', 'name': 'BTPOOL'}),
  ('/Rawpool.com/',
   {'link': 'https://www.rawpool.com/', 'name': 'Rawpool.com'}),
  ('/Helix/', {'link': '', 'name': 'Helix'}),
  ('/Bitcoin-Ukraine.com.ua/',
   {'link': 'https://bitcoin-ukraine.com.ua/', 'name': 'Bitcoin-Ukraine'}),
  ('/poolin.com', {'link': 'https://www.poolin.com/', 'name': 'Poolin'}),
  ('/SecretSuperstar/', {'link': '', 'name': 'SecretSuperstar'}),
  ('/tigerpool.net', {'link

In [53]:
for diff in diffs:
    if diff[0] == "change":
        #if type(diff[1]) is list:
        #    "name" in diff[1]:
        #        pprint.pprint(diff)
        #else:
            if "name" in diff[1]:
                pprint.pprint(diff)

('change',
 ['coinbase_tags', '/solo.ckpool.org/', 'name'],
 ('Solo CKPool', 'Solo CK'))
('change', 'coinbase_tags./NiceHashSolo.name', ('NiceHash Solo', 'NiceHash'))
('change',
 'coinbase_tags./BitClub Network/.name',
 ('BitClub Network', 'BitClub'))
('change', 'coinbase_tags.BTCChina Pool.name', ('BTCC Pool', 'BTCC'))
('change', ['coinbase_tags', 'btcchina.com', 'name'], ('BTCC Pool', 'BTCC'))
('change', ['coinbase_tags', 'BTCChina.com', 'name'], ('BTCC Pool', 'BTCC'))
('change', 'coinbase_tags./BTCC/.name', ('BTCC Pool', 'BTCC'))
('change', 'coinbase_tags.BW Pool.name', ('BW.COM', 'BWPool'))
('change',
 ['coinbase_tags', 'xbtc.exx.com&bw.com', 'name'],
 ('xbtc.exx.com&bw.com', 'EXX&BW'))
('change', 'coinbase_tags./CANOE/.name', ('CANOE', 'CanoePool'))
('change', 'coinbase_tags./haominer/.name', ('Haominer', 'haominer'))
('change',
 'payout_addresses.1JLRXD8rjRgQtTS9MvfQALfHgGWau9L9ky.name',
 ('BW.COM', 'BWPool'))
('change',
 'payout_addresses.155fzsEBHy9Ri2bMQ8uuuR3tv1YzcDywd4.name'

## Create miners\_initial\_....json file

In [54]:
class ConflictingPoolData(Exception):
    pass

def add_makers_from_source(pools_dict,miners_dict,source,currency=[ "BTC",] ):
    # Add all markers from blockchain.info pools.json style file converted to a pools_dict
    # fields of blockchain.info dict
    fpayout="payout_addresses"
    ftags="coinbase_tags"
    fname="name"
    furl="link"

    for marker in pools_dict[ftags]:
        pool_name = pools_dict[ftags][marker][fname]
        pool_name_lookup = util.get_miner_id_by_name(miners_dict,pool_name)
        if pool_name_lookup is not None and pool_name_lookup != pool_name:
            # print a note if a pool has more than one name, this is ok 
            print("duplicate name in markers: " + pool_name + " - " + pool_name_lookup)
            pool_name = pool_name_lookup
        pool_url = pools_dict[ftags][marker][furl]

        
        if pool_name in miners_dict.keys() and marker in miners_dict[ pool_name ][ util.D_MARKERS ]:
            # print a note if a pool has more than one marker, this is ok 
            print("Same marker for pool: " + pool_name + " marker:" +  marker)
        elif pool_name in miners_dict.keys():
            # print a note if a pool has more than one marker, this is ok 
            print("Multiple markers for pool: " + pool_name)

        names_dict = { pools_dict[ftags][marker][fname]: { util.DD_URL:pool_url,
                                    util.DD_CURRENCIES: currency,
                                    util.DD_FULLNAME: "",
                                    util.DD_FIRSTUSED: 0,
                                    util.DD_LASTUSED: 0,
                                    util.DD_SOURCES: source} } 

        markers_dict = { marker: {  util.DD_CURRENCIES: currency,
                                    util.DD_FIRSTUSED: 0,
                                    util.DD_LASTUSED: 0,
                                    util.DD_SOURCES: source } }  

        util.add_miner(pool_name,
                 miners_dict,
                 names_dict,
                 markers_dict)

        #print(pool_name + "\t" + marker + "\t" + pool_url)
    return miners_dict


In [55]:
def add_addresses_from_source(pools_dict,miners_dict,source,currency=[ "BTC",] ):
    # Add all addresses from blockchain.info style file 
    # fields of blockchain.info dict
    fpayout="payout_addresses"
    ftags="coinbase_tags"
    fname="name"
    furl="link"
    
    for address in pools_dict[fpayout]:
        pool_name = pools_dict[fpayout][address][fname]
        pool_url = pools_dict[fpayout][address][furl]
        
        pool_name_lookup = util.get_miner_id_by_name(miners_dict,pool_name)
        if pool_name_lookup is not None and pool_name_lookup != pool_name:
            # print a note if a pool has more than one name, this is ok 
            print("duplicate name in addresses: " + pool_name + " - " + pool_name_lookup)
            pool_name = pool_name_lookup
        
        if pool_name in miners_dict:
            # add address to a existing pool
            #print("Adding address " + address + " to pool "+ pool_name)

            # check for duplicated pool_names with different addresses, this should not happen
            #print(json.dumps(miners, indent=2, sort_keys=True))
            stored_pool_url = miners_dict[ pool_name ][ util.D_NAMES ][ pool_name ]["url"]

            if ( pool_url.strip("/").strip("https://www.").strip("http://www.").strip("http://")
            != stored_pool_url.strip("/").strip("https://www.").strip("http://www.").strip("http://") ):
                print("CONFLICTING URL: " + repr(pool_url) )
                print("CONFLICTING URL: " + repr(stored_pool_url) )
                #raise ConflictingPoolData()

            # check if address already added 
            #if address in miners[pool_name]["addresses"]:
            #    print(" " + repr(pool_url) + repr(pool_name) + repr(address))
            #    print(" " + repr(miners[pool_name]["addresses"][address]) )
            #    raise ConflictingPoolData()
                
            names_dict = { pools_dict[fpayout][address][fname]: { util.DD_URL:pool_url,
                                    util.DD_CURRENCIES: currency,
                                    util.DD_FULLNAME: "",
                                    util.DD_FIRSTUSED: 0,
                                    util.DD_LASTUSED: 0,
                                    util.DD_SOURCES: source} }
            
            addresses_dict={ address: { util.DD_CURRENCIES: currency,
                                                 util.DD_FIRSTUSED: 0,
                                                 util.DD_LASTUSED: 0,
                                                 util.DD_SOURCES: source } }

            util.add_miner(miner_id=pool_name,
                     miners=miners_dict,
                     names_dict=names_dict,
                     addresses_dict=addresses_dict)
        else:
            # ceate new pool for new address
            print("new pool: " + pool_name)
            util.add_miner(miner_id=pool_name,
                     miners=miners_dict,
                     names_dict= { pool_name: { util.DD_URL:pool_url,
                                                util.DD_CURRENCIES: currency,
                                                util.DD_FULLNAME: "",
                                                util.DD_FIRSTUSED: 0,
                                                util.DD_LASTUSED: 0,
                                                util.DD_SOURCES:source } },
                     addresses_dict= { address: { util.DD_CURRENCIES: currency,
                                                  util.DD_FIRSTUSED: 0,
                                                  util.DD_LASTUSED: 0,
                                                  util.DD_SOURCES: source } }, )
    return miners_dict
        

## Create miners_initial_blockchaininfo.json file from blockchain.info
This initial pools files is based soly on data from *blockchain.info* github repository. 
It should give us a clear start to begin with. 

In [56]:
# Boostrap pools files with blockchain.info github data
# https://github.com/blockchain/Blockchain-Known-Pools/tree/82ed31956388e3950845cc2faeaf6679a057ee5b
miners_initial_blockchaininfo = dict()
miners_initial_blockchaininfo.clear()

miners_initial_blockchaininfo = add_makers_from_source(pools_bc,miners_dict=miners_initial_blockchaininfo,source=["blockchain.info github",])

Multiple markers for pool: OzCoin
Multiple markers for pool: TripleMining
Multiple markers for pool: Polmine
Multiple markers for pool: AntPool
Multiple markers for pool: BTCC Pool
Multiple markers for pool: BTCC Pool
Multiple markers for pool: BTCC Pool
Multiple markers for pool: BitFury
Multiple markers for pool: ViaBTC
Multiple markers for pool: PHash.IO


In [57]:
miners_initial_blockchaininfo = add_addresses_from_source(pools_bc,miners_dict=miners_initial_blockchaininfo,source=["blockchain.info github",])

new pool: BTC Nuggets
new pool: EkanemBTC
new pool: Huobi
new pool: CloudHashing
new pool: digitalX Mintsy
new pool: Telco 214
new pool: BTC Pool Party
new pool: Multipool
new pool: transactioncoinmining
new pool: BTCDig
new pool: Tricky's BTC Pool
new pool: BTCMP
new pool: Eobot
new pool: UNOMP
new pool: Patel's Mining pool
new pool: GoGreenLight
new pool: Poolin


In [58]:
# 'digitalX Mintsy' => 'digitalBTC'
util.add_miner('digitalBTC',miners_initial_blockchaininfo) # already exists 
util.unify_miners('digitalX Mintsy','digitalBTC',miners_initial_blockchaininfo)
print()

In [59]:
util.get_sample(miners_initial_blockchaininfo)

{'names': {'Mt Red': {'url': 'https://mtred.com',
   'currencies': ['BTC'],
   'fullName': '',
   'firstUsed': 0,
   'lastUsed': 0,
   'sources': ['blockchain.info github']}},
 'markers': {'/mtred/': {'currencies': ['BTC'],
   'firstUsed': 0,
   'lastUsed': 0,
   'sources': ['blockchain.info github']}},
 'addresses': {}}

In [60]:
with open(miners_initial_blockchaininfo_json_file, 'w') as outfile:
    json.dump(miners_initial_blockchaininfo, outfile)

## blockchain.info only attribution

In [61]:
# freshly read clean blocks.json file without any attributions
blocks = dict()
blocks.clear()

with open(blocks_json_file, 'r') as fp:
    blocks = json.load(fp)

In [62]:
# freshly read lean miners_minitial file (from blockchain.info) without any modifications
miners_initial_blockchaininfo = dict()
miners_initial_blockchaininfo.clear()

with open(miners_initial_blockchaininfo_json_file, 'r') as fp:
    miners_initial_blockchaininfo = json.load(fp)

In [63]:
util.get_sample(miners_initial_blockchaininfo)

{'names': {'Waterhole': {'url': 'https://btc.waterhole.io/',
   'currencies': ['BTC'],
   'fullName': '',
   'firstUsed': 0,
   'lastUsed': 0,
   'sources': ['blockchain.info github']}},
 'markers': {'/WATERHOLE.IO/': {'currencies': ['BTC'],
   'firstUsed': 0,
   'lastUsed': 0,
   'sources': ['blockchain.info github']}},
 'addresses': {'1FLH1SoLv4U68yUERhDiWzrJn5TggMqkaZ': {'currencies': ['BTC'],
   'firstUsed': 0,
   'lastUsed': 0,
   'sources': ['blockchain.info github']}}}

In [64]:
def attribute_blocks(blocks,
                     miners_dict,
                     addr_attr,
                     marker_attr,
                     both_attr,
                     source,
                     override=False,
                     update=False):
    """ Attribute given blocks based on given miners_dict json
    
    Takes names for the different attribution per address, marker and both as well as a source 
    from where the miners_dict information comes from. Overrides existing attributions with given
    names if override flag is set. 
    Returns tuple of (blocks,miners_dict,conflicts) and does change miners_dict in the process.
    """
    i = 0
    conflicts = list()
    conflicts.clear()

    for blknum in blocks:
        match = list()
        addr_match = list()
        cb_match = list()

        try:
            # first always test if not already attributed 
            if ( addr_attr not in blocks[ blknum ][ util.D_ATTRIBUTIONS ].keys() ) or override:
                # match address
                if len( blocks[ blknum ][ util.D_ADDRESSES ] ) == 1:
                    # only match if there is just one output address in the coinbase 
                    address = blocks[ blknum ][ util.D_ADDRESSES ][0]
                    match = util.match_address_to_miner( address, miners_dict, strict=False, blknum=int(blknum) )

                    if len( match ) >= 1:
                        # if multiple coinbase markers match we can get more than one match
                        matched_miners = defaultdict(list)
                        for ma in match:
                            matched_miners[ ma[0] ].append( ma[1] )
                        j = 0
                        attr = ""
                        for mi in matched_miners:
                            blocks[ blknum ][ util.D_ATTRIBUTIONS ][ addr_attr + attr ] = { util.DDD_MINER:mi,
                                                                                               "matches":matched_miners[mi],
                                                                                               util.DDD_SRC:source }
                            j += 1
                            attr = str(j)

            if ( marker_attr not in blocks[ blknum ][ util.D_ATTRIBUTIONS ].keys() ) or override:
                # match coinbase
                coinbase = blocks[ blknum ][ util.D_CB ]
                match = util.match_coinbase_to_miner( coinbase, miners_dict, strict=False, blknum=int(blknum) )

                if len( match ) >= 1:
                    # if multiple coinbase markers match we can get more than one match
                    matched_miners = defaultdict(list)
                    for ma in match:
                        matched_miners[ ma[0] ].append( ma[1] )
                    j = 0
                    attr = ""
                    for mi in matched_miners:
                        blocks[ blknum ][ util.D_ATTRIBUTIONS ][ marker_attr + attr ] = { util.DDD_MINER:mi,
                                                                                           "matches":matched_miners[mi],
                                                                                           util.DDD_SRC:source }
                        j += 1
                        attr = str(j)

            if ( both_attr not in blocks[ blknum ][ util.D_ATTRIBUTIONS ].keys() ) or override:
                # match both and update miners
                coinbase = blocks[ blknum ][ util.D_CB ]
                if len( blocks[ blknum ][ util.D_ADDRESSES ] ) == 1:
                    address = blocks[ blknum ][ util.D_ADDRESSES ][0]
                    match = util.match_miner(miners_dict,address,coinbase,update=update, blknum=int(blknum) )
                else:
                    match = util.match_miner(miners=miners_dict,coinbase=coinbase, blknum=int(blknum) )

                if len( match ) > 0:
                    # There could be more than one marker of the same pool that matches simultaniously
                    matches = list()
                    #print(match)
                    for m in match:
                        matches.append( m[1] )
                    blocks[ blknum ][ util.D_ATTRIBUTIONS ][ both_attr ] = { util.DDD_MINER:match[0][0],
                                                                                    "matches":matches,
                                                                                    util.DDD_SRC:source }
        except util.ConflictingMinerData as e:
            print()
            print("Message    = ",e.message)
            print("Blockheight= ",blknum)
            print("Miner1     = ",e.miner1)
            print("Miner2     = ",e.miner2)
            print("Coinbase   = ",e.coinbase)
            print("CoinbaseStr= ",repr(binascii.unhexlify(e.coinbase)))
            print("Addesses   = ",e.address)
            print("addr_match = ",e.addr_match)
            print("cb_match   = ",e.cb_match)
            conflicts.append( { "message":e.message,
                                util.DDD_MINER + "1":e.miner1,
                                util.DDD_MINER + "2":e.miner2,
                                util.D_CB + "1":e.coinbase,
                                "address": e.address,
                                "addr_match": e.addr_match,
                                "cb_match": e.cb_match,
                                util.DDD_SRC:source } )

        # progress bar     
        i+=1
        if i % 1000 == 0:
            print(i,end="\r")
            sys.stdout.flush()
    return (blocks,miners_dict,conflicts)

In [65]:
# this dict will be updated when coinbase matches are found and there is only one address 
# as the coinbase output
#miners = dict()
#miners.clear()
#miners = copy.deepcopy(miners_initial_blockchaininfo)

Attribute blocks to miners according to **blockchain.info** github file.
Even this file contains some conflicts when attributing according to address vs marker.
See below for conflicts:

In [66]:
# attribute blocks to miners according to blockcahin.info initial mapping
(blocks,miners_initial_blockchaininfo,conflicts) = util.attribute_blocks(blocks=blocks,
                                   miners_dict=miners_initial_blockchaininfo,
                                   addr_attr=util.DD_BCI_ADDR_ATTR,
                                   marker_attr=util.DD_BCI_MARK_ATTR,
                                   both_attr=util.DD_BCI_ATTR,
                                   source="blockchain.info",
                                   override=True)

482000
Message    =  Addr and Cb match differ
Blockheight=  482059
Miner1     =  Waterhole
Miner2     =  BTC.com
Coinbase   =  030b5b0704465fa1592f4254432e434f4d2ffabe6d6dff48161efa0f0b44fc9463e380e6b585ff91ac3f9c973d156dfff35fd553cdb101000000000000000228124b7faf010000000000
CoinbaseStr=  b'\x03\x0b[\x07\x04F_\xa1Y/BTC.COM/\xfa\xbemm\xffH\x16\x1e\xfa\x0f\x0bD\xfc\x94c\xe3\x80\xe6\xb5\x85\xff\x91\xac?\x9c\x97=\x15m\xff\xf3_\xd5S\xcd\xb1\x01\x00\x00\x00\x00\x00\x00\x00\x02(\x12K\x7f\xaf\x01\x00\x00\x00\x00\x00'
Addesses   =  1FLH1SoLv4U68yUERhDiWzrJn5TggMqkaZ
addr_match =  [('Waterhole', {'addr_match': '1FLH1SoLv4U68yUERhDiWzrJn5TggMqkaZ'})]
cb_match   =  [('BTC.com', {'cb_match': '/BTC.COM/'})]

Message    =  Addr and Cb match differ
Blockheight=  482221
Miner1     =  Waterhole
Miner2     =  BTC.com
Coinbase   =  03ad5b0704a4e6a2592f4254432e434f4d2ffabe6d6d0dd4f6c7dac894c84821c9fed8c15a621583ada8e4177c182c011cab52e2d6d0010000000000000001292f8e92ae010000000000
CoinbaseStr=  b'\x03\xad[\x

In [67]:
len(conflicts) # 2

2

In [68]:
# introduced checkpoint
assert 2 == len(conflicts)

In [69]:
# attribute blocks to miners according to blockcahin.info initial mapping
(blocks,miners_initial_blockchaininfo,conflicts) = util.attribute_blocks(blocks=blocks,
                                   miners_dict=miners_initial_blockchaininfo,
                                   addr_attr=util.DD_BCI_ADDR_ATTR + "_update",
                                   marker_attr=util.DD_BCI_MARK_ATTR + "_update",
                                   both_attr=util.DD_BCI_ATTR + "_update",
                                   source="blockchain.info",
                                   override=True,
                                   update=True)

160000
Message    =  Addr and Cb match differ
Blockheight=  160367
Miner1     =  OzCoin
Miner2     =  Yourbtc.net
Coinbase   =  70736a04ba760e1a046c730300522cfabe6d6dad6ee140dbdc015c8da85b8e3b636dc896e7df9203a7cd2c41e858503563ac130100000000000000796f75726274632e6e6574ac1eeeed88
CoinbaseStr=  b'psj\x04\xbav\x0e\x1a\x04ls\x03\x00R,\xfa\xbemm\xadn\xe1@\xdb\xdc\x01\\\x8d\xa8[\x8e;cm\xc8\x96\xe7\xdf\x92\x03\xa7\xcd,A\xe8XP5c\xac\x13\x01\x00\x00\x00\x00\x00\x00\x00yourbtc.net\xac\x1e\xee\xed\x88'
Addesses   =  1GG9HQZchCRxPSBV5SwZ9GoYEVq9vVLGqU
addr_match =  [('OzCoin', {'addr_match': '1GG9HQZchCRxPSBV5SwZ9GoYEVq9vVLGqU'})]
cb_match   =  [('Yourbtc.net', {'cb_match': 'yourbtc.net'})]

Message    =  Addr and Cb match differ
Blockheight=  160464
Miner1     =  OzCoin
Miner2     =  Yourbtc.net
Coinbase   =  70736a04ba760e1a04136c1100522cfabe6d6d7d24b129b8b97a267e531f9d816c6f435c9bb8a5fcc78b83360a5fee452c9e3e0100000000000000796f75726274632e6e6574ac1eeeed88
CoinbaseStr=  b'psj\x04\xbav\x0e\x1a\x04

162000
Message    =  Addr and Cb match differ
Blockheight=  162024
Miner1     =  OzCoin
Miner2     =  Yourbtc.net
Coinbase   =  70736a04d7690d1a04f9aa3005522cfabe6d6d079ca0876cf5e21bfbdbae1b7a75b2aae290cebfe0493d57935c37250cb145460100000000000000796f75726274632e6e6574ac1eeeed88
CoinbaseStr=  b'psj\x04\xd7i\r\x1a\x04\xf9\xaa0\x05R,\xfa\xbemm\x07\x9c\xa0\x87l\xf5\xe2\x1b\xfb\xdb\xae\x1bzu\xb2\xaa\xe2\x90\xce\xbf\xe0I=W\x93\\7%\x0c\xb1EF\x01\x00\x00\x00\x00\x00\x00\x00yourbtc.net\xac\x1e\xee\xed\x88'
Addesses   =  1GG9HQZchCRxPSBV5SwZ9GoYEVq9vVLGqU
addr_match =  [('OzCoin', {'addr_match': '1GG9HQZchCRxPSBV5SwZ9GoYEVq9vVLGqU'})]
cb_match   =  [('Yourbtc.net', {'cb_match': 'yourbtc.net'})]

Message    =  Addr and Cb match differ
Blockheight=  162033
Miner1     =  OzCoin
Miner2     =  Yourbtc.net
Coinbase   =  70736a04d7690d1a04de9f4305522cfabe6d6dddd07348993994c5fef67c13a320f3303babe98124f79768eb3fe51626b66ae90100000000000000796f75726274632e6e6574ac1eeeed88
CoinbaseStr=  b'psj\x04\xd7i\r\x1


Message    =  Addr and Cb match differ
Blockheight=  162549
Miner1     =  OzCoin
Miner2     =  Yourbtc.net
Coinbase   =  70736a04d7690d1a0426d74000522cfabe6d6dec35356dd27d3b7b8b06a5370fb62ce9a0d265eb20e4490a29915b3b7a4619840100000000000000796f75726274632e6e6574ac1eeeed88
CoinbaseStr=  b'psj\x04\xd7i\r\x1a\x04&\xd7@\x00R,\xfa\xbemm\xec55m\xd2};{\x8b\x06\xa57\x0f\xb6,\xe9\xa0\xd2e\xeb \xe4I\n)\x91[;zF\x19\x84\x01\x00\x00\x00\x00\x00\x00\x00yourbtc.net\xac\x1e\xee\xed\x88'
Addesses   =  1GG9HQZchCRxPSBV5SwZ9GoYEVq9vVLGqU
addr_match =  [('OzCoin', {'addr_match': '1GG9HQZchCRxPSBV5SwZ9GoYEVq9vVLGqU'})]
cb_match   =  [('Yourbtc.net', {'cb_match': 'yourbtc.net'})]

Message    =  Addr and Cb match differ
Blockheight=  162576
Miner1     =  OzCoin
Miner2     =  Yourbtc.net
Coinbase   =  70736a04d7690d1a042d349e00522cfabe6d6d512a6f5db63e748705a3f42980066729291c0fea939c087c737921e6a0abca430100000000000000796f75726274632e6e6574ac1eeeed88
CoinbaseStr=  b'psj\x04\xd7i\r\x1a\x04-4\x9e\x00R,\xfa\xbemm

Addesses   =  1GG9HQZchCRxPSBV5SwZ9GoYEVq9vVLGqU
addr_match =  [('OzCoin', {'addr_match': '1GG9HQZchCRxPSBV5SwZ9GoYEVq9vVLGqU'})]
cb_match   =  [('Yourbtc.net', {'cb_match': 'yourbtc.net'})]

Message    =  Addr and Cb match differ
Blockheight=  163129
Miner1     =  OzCoin
Miner2     =  Yourbtc.net
Coinbase   =  70736a04d7690d1a04e1f77b00522cfabe6d6d1b13a18e0306652c06f408d8c920e05ee97bf61a71899b7bd446a8e1f65c42150100000000000000796f75726274632e6e6574ac1eeeed88
CoinbaseStr=  b'psj\x04\xd7i\r\x1a\x04\xe1\xf7{\x00R,\xfa\xbemm\x1b\x13\xa1\x8e\x03\x06e,\x06\xf4\x08\xd8\xc9 \xe0^\xe9{\xf6\x1aq\x89\x9b{\xd4F\xa8\xe1\xf6\\B\x15\x01\x00\x00\x00\x00\x00\x00\x00yourbtc.net\xac\x1e\xee\xed\x88'
Addesses   =  1GG9HQZchCRxPSBV5SwZ9GoYEVq9vVLGqU
addr_match =  [('OzCoin', {'addr_match': '1GG9HQZchCRxPSBV5SwZ9GoYEVq9vVLGqU'})]
cb_match   =  [('Yourbtc.net', {'cb_match': 'yourbtc.net'})]

Message    =  Addr and Cb match differ
Blockheight=  163145
Miner1     =  OzCoin
Miner2     =  Yourbtc.net
Coinbase  

In [70]:
(blocks,miners_initial_blockchaininfo,conflicts) = util.attribute_blocks(blocks=blocks,
                                   miners_dict=miners_initial_blockchaininfo,
                                   addr_attr=util.DD_BCI_ADDR_ATTR + "_update",
                                   marker_attr=util.DD_BCI_MARK_ATTR + "_update",
                                   both_attr=util.DD_BCI_ATTR + "_update",
                                   source="blockchain.info",
                                   override=True,
                                   update=True)

160000
Message    =  Addr and Cb match differ
Blockheight=  160367
Miner1     =  OzCoin
Miner2     =  Yourbtc.net
Coinbase   =  70736a04ba760e1a046c730300522cfabe6d6dad6ee140dbdc015c8da85b8e3b636dc896e7df9203a7cd2c41e858503563ac130100000000000000796f75726274632e6e6574ac1eeeed88
CoinbaseStr=  b'psj\x04\xbav\x0e\x1a\x04ls\x03\x00R,\xfa\xbemm\xadn\xe1@\xdb\xdc\x01\\\x8d\xa8[\x8e;cm\xc8\x96\xe7\xdf\x92\x03\xa7\xcd,A\xe8XP5c\xac\x13\x01\x00\x00\x00\x00\x00\x00\x00yourbtc.net\xac\x1e\xee\xed\x88'
Addesses   =  1GG9HQZchCRxPSBV5SwZ9GoYEVq9vVLGqU
addr_match =  [('OzCoin', {'addr_match': '1GG9HQZchCRxPSBV5SwZ9GoYEVq9vVLGqU'})]
cb_match   =  [('Yourbtc.net', {'cb_match': 'yourbtc.net'})]

Message    =  Addr and Cb match differ
Blockheight=  160464
Miner1     =  OzCoin
Miner2     =  Yourbtc.net
Coinbase   =  70736a04ba760e1a04136c1100522cfabe6d6d7d24b129b8b97a267e531f9d816c6f435c9bb8a5fcc78b83360a5fee452c9e3e0100000000000000796f75726274632e6e6574ac1eeeed88
CoinbaseStr=  b'psj\x04\xbav\x0e\x1a\x04


Message    =  Addr and Cb match differ
Blockheight=  161881
Miner1     =  OzCoin
Miner2     =  Yourbtc.net
Coinbase   =  70736a04d7690d1a040dbcaa01522cfabe6d6d3f8d5d406aacef2699b5b8001c12df9186b6b27846b53815b07a5370771874700100000000000000796f75726274632e6e6574ac1eeeed88
CoinbaseStr=  b'psj\x04\xd7i\r\x1a\x04\r\xbc\xaa\x01R,\xfa\xbemm?\x8d]@j\xac\xef&\x99\xb5\xb8\x00\x1c\x12\xdf\x91\x86\xb6\xb2xF\xb58\x15\xb0zSpw\x18tp\x01\x00\x00\x00\x00\x00\x00\x00yourbtc.net\xac\x1e\xee\xed\x88'
Addesses   =  1GG9HQZchCRxPSBV5SwZ9GoYEVq9vVLGqU
addr_match =  [('OzCoin', {'addr_match': '1GG9HQZchCRxPSBV5SwZ9GoYEVq9vVLGqU'})]
cb_match   =  [('Yourbtc.net', {'cb_match': 'yourbtc.net'})]

Message    =  Addr and Cb match differ
Blockheight=  161899
Miner1     =  OzCoin
Miner2     =  Yourbtc.net
Coinbase   =  70736a04d7690d1a042e70cc01522cfabe6d6d20dcbd6fec99828e60aea932c70bbf272de0ac34f88fd37885af139b2a1511f40100000000000000796f75726274632e6e6574ac1eeeed88
CoinbaseStr=  b"psj\x04\xd7i\r\x1a\x04.p\xcc\x01

Blockheight=  162205
Miner1     =  OzCoin
Miner2     =  Yourbtc.net
Coinbase   =  70736a04d7690d1a045ef95006522cfabe6d6da36c2d9a23155dd5426177ac77a0e4e218f7fb5ff9d6dc8e9ae6b19c4eb574a40100000000000000796f75726274632e6e6574ac1eeeed88
CoinbaseStr=  b'psj\x04\xd7i\r\x1a\x04^\xf9P\x06R,\xfa\xbemm\xa3l-\x9a#\x15]\xd5Baw\xacw\xa0\xe4\xe2\x18\xf7\xfb_\xf9\xd6\xdc\x8e\x9a\xe6\xb1\x9cN\xb5t\xa4\x01\x00\x00\x00\x00\x00\x00\x00yourbtc.net\xac\x1e\xee\xed\x88'
Addesses   =  1GG9HQZchCRxPSBV5SwZ9GoYEVq9vVLGqU
addr_match =  [('OzCoin', {'addr_match': '1GG9HQZchCRxPSBV5SwZ9GoYEVq9vVLGqU'})]
cb_match   =  [('Yourbtc.net', {'cb_match': 'yourbtc.net'})]

Message    =  Addr and Cb match differ
Blockheight=  162221
Miner1     =  OzCoin
Miner2     =  Yourbtc.net
Coinbase   =  70736a04d7690d1a0429c62b00522cfabe6d6d6aa88a20b82c6e273be1575f5ac763eaea97d578ee8a0b5802da191932b98bc40100000000000000796f75726274632e6e6574ac1eeeed88
CoinbaseStr=  b"psj\x04\xd7i\r\x1a\x04)\xc6+\x00R,\xfa\xbemmj\xa8\x8a \xb8,n';\xe1W


Message    =  Addr and Cb match differ
Blockheight=  162746
Miner1     =  OzCoin
Miner2     =  Yourbtc.net
Coinbase   =  70736a04d7690d1a04591f1400522cfabe6d6d0b1de5aab4e9f2be360cdd841c6c14e048c214d2c6c9e01e06e9a352502b48e70100000000000000796f75726274632e6e6574ac1eeeed88
CoinbaseStr=  b'psj\x04\xd7i\r\x1a\x04Y\x1f\x14\x00R,\xfa\xbemm\x0b\x1d\xe5\xaa\xb4\xe9\xf2\xbe6\x0c\xdd\x84\x1cl\x14\xe0H\xc2\x14\xd2\xc6\xc9\xe0\x1e\x06\xe9\xa3RP+H\xe7\x01\x00\x00\x00\x00\x00\x00\x00yourbtc.net\xac\x1e\xee\xed\x88'
Addesses   =  1GG9HQZchCRxPSBV5SwZ9GoYEVq9vVLGqU
addr_match =  [('OzCoin', {'addr_match': '1GG9HQZchCRxPSBV5SwZ9GoYEVq9vVLGqU'})]
cb_match   =  [('Yourbtc.net', {'cb_match': 'yourbtc.net'})]

Message    =  Addr and Cb match differ
Blockheight=  162778
Miner1     =  OzCoin
Miner2     =  Yourbtc.net
Coinbase   =  70736a04d7690d1a048b664e00522cfabe6d6d42179d6ed820e16af5461986c134cd013fd6991e5f0b9ff8d7b96ccc8aaa66300100000000000000796f75726274632e6e6574ac1eeeed88
CoinbaseStr=  b'psj\x04\xd7i

482000
Message    =  Addr and Cb match differ
Blockheight=  482059
Miner1     =  Waterhole
Miner2     =  BTC.com
Coinbase   =  030b5b0704465fa1592f4254432e434f4d2ffabe6d6dff48161efa0f0b44fc9463e380e6b585ff91ac3f9c973d156dfff35fd553cdb101000000000000000228124b7faf010000000000
CoinbaseStr=  b'\x03\x0b[\x07\x04F_\xa1Y/BTC.COM/\xfa\xbemm\xffH\x16\x1e\xfa\x0f\x0bD\xfc\x94c\xe3\x80\xe6\xb5\x85\xff\x91\xac?\x9c\x97=\x15m\xff\xf3_\xd5S\xcd\xb1\x01\x00\x00\x00\x00\x00\x00\x00\x02(\x12K\x7f\xaf\x01\x00\x00\x00\x00\x00'
Addesses   =  1FLH1SoLv4U68yUERhDiWzrJn5TggMqkaZ
addr_match =  [('Waterhole', {'addr_match': '1FLH1SoLv4U68yUERhDiWzrJn5TggMqkaZ'})]
cb_match   =  [('BTC.com', {'cb_match': '/BTC.COM/'})]

Message    =  Addr and Cb match differ
Blockheight=  482221
Miner1     =  Waterhole
Miner2     =  BTC.com
Coinbase   =  03ad5b0704a4e6a2592f4254432e434f4d2ffabe6d6d0dd4f6c7dac894c84821c9fed8c15a621583ada8e4177c182c011cab52e2d6d0010000000000000001292f8e92ae010000000000
CoinbaseStr=  b'\x03\xad[\x

In [71]:
len(conflicts) # 5

65

In [72]:
with open(miners_initial_blockchaininfo_conflicts_json_file, 'w') as outfile:
    json.dump(conflicts, outfile)

In [73]:
blocks["353757"]

{'time': 1430024584,
 'cb': '03dd6505062f503253482f047c713c5508fabe6d6d00009582bc1f01000000002f6d75746172745365646f6e2f0dd38a2f05e3ffff010000000000000077ffffe5053d8c490d2f6e6f64655374726174756d2f',
 'addresses': ['1NY15MK947MLzmPUa2gL7UgyR8prLh2xfu'],
 'miner': '',
 'conflicts': 0,
 'attribution': '',
 'attributions': {'blockchain_info_address': {'miner': 'digitalBTC',
   'matches': [{'addr_match': '1NY15MK947MLzmPUa2gL7UgyR8prLh2xfu'}],
   'src': 'blockchain.info'},
  'blockchain_info': {'miner': 'digitalBTC',
   'matches': [{'addr_match': '1NY15MK947MLzmPUa2gL7UgyR8prLh2xfu'}],
   'src': 'blockchain.info'},
  'blockchain_info_address_update': {'miner': 'digitalBTC',
   'matches': [{'addr_match': '1NY15MK947MLzmPUa2gL7UgyR8prLh2xfu'}],
   'src': 'blockchain.info'},
  'blockchain_info_update': {'miner': 'digitalBTC',
   'matches': [{'addr_match': '1NY15MK947MLzmPUa2gL7UgyR8prLh2xfu'}],
   'src': 'blockchain.info'}},
 'hash': '00000000000000000e6ed6f2db8296aab547670f4cf23847fec76161723e

Print and investigate the conflicts:

In [74]:
blocks["482059"]

{'time': 1503747909,
 'cb': '030b5b0704465fa1592f4254432e434f4d2ffabe6d6dff48161efa0f0b44fc9463e380e6b585ff91ac3f9c973d156dfff35fd553cdb101000000000000000228124b7faf010000000000',
 'addresses': ['1FLH1SoLv4U68yUERhDiWzrJn5TggMqkaZ'],
 'miner': '',
 'conflicts': 0,
 'attribution': '',
 'attributions': {'blockchain_info_address': {'miner': 'Waterhole',
   'matches': [{'addr_match': '1FLH1SoLv4U68yUERhDiWzrJn5TggMqkaZ'}],
   'src': 'blockchain.info'},
  'blockchain_info_marker': {'miner': 'BTC.com',
   'matches': [{'cb_match': '/BTC.COM/'}],
   'src': 'blockchain.info'},
  'blockchain_info_address_update': {'miner': 'Waterhole',
   'matches': [{'addr_match': '1FLH1SoLv4U68yUERhDiWzrJn5TggMqkaZ'}],
   'src': 'blockchain.info'},
  'blockchain_info_marker_update': {'miner': 'BTC.com',
   'matches': [{'cb_match': '/BTC.COM/'}],
   'src': 'blockchain.info'}},
 'hash': '000000000000000000ee874c85580ea9c9297d4fcf4fe0bf59b696d1dbd10290',
 'phash': None,
 'payout': '1701199633'}

In [75]:
blocks["482221"]

{'time': 1503848099,
 'cb': '03ad5b0704a4e6a2592f4254432e434f4d2ffabe6d6d0dd4f6c7dac894c84821c9fed8c15a621583ada8e4177c182c011cab52e2d6d0010000000000000001292f8e92ae010000000000',
 'addresses': ['1FLH1SoLv4U68yUERhDiWzrJn5TggMqkaZ'],
 'miner': '',
 'conflicts': 0,
 'attribution': '',
 'attributions': {'blockchain_info_address': {'miner': 'Waterhole',
   'matches': [{'addr_match': '1FLH1SoLv4U68yUERhDiWzrJn5TggMqkaZ'}],
   'src': 'blockchain.info'},
  'blockchain_info_marker': {'miner': 'BTC.com',
   'matches': [{'cb_match': '/BTC.COM/'}],
   'src': 'blockchain.info'},
  'blockchain_info_address_update': {'miner': 'Waterhole',
   'matches': [{'addr_match': '1FLH1SoLv4U68yUERhDiWzrJn5TggMqkaZ'}],
   'src': 'blockchain.info'},
  'blockchain_info_marker_update': {'miner': 'BTC.com',
   'matches': [{'cb_match': '/BTC.COM/'}],
   'src': 'blockchain.info'}},
 'hash': '000000000000000000d172ef46944db6127dbebe815664f26f37fef3e22fd65b',
 'phash': None,
 'payout': '1293786138'}

In [76]:
blocks["159929"]

{'time': 1325312734,
 'cb': '70736a04ba760e1a0418930700522cfabe6d6dc15059fd58be57a512008aff27237100949e3afd1e7f7be1dc5cc5346f7169b401000000000000006f7a636f2e696eac1eeeed88',
 'addresses': ['1GG9HQZchCRxPSBV5SwZ9GoYEVq9vVLGqU'],
 'miner': '',
 'conflicts': 0,
 'attribution': '',
 'attributions': {'blockchain_info_marker': {'miner': 'OzCoin',
   'matches': [{'cb_match': 'ozco.in'}],
   'src': 'blockchain.info'},
  'blockchain_info': {'miner': 'OzCoin',
   'matches': [{'cb_match': 'ozco.in'}],
   'src': 'blockchain.info'},
  'blockchain_info_address_update': {'miner': 'OzCoin',
   'matches': [{'addr_match': '1GG9HQZchCRxPSBV5SwZ9GoYEVq9vVLGqU'}],
   'src': 'blockchain.info'},
  'blockchain_info_marker_update': {'miner': 'OzCoin',
   'matches': [{'cb_match': 'ozco.in'}],
   'src': 'blockchain.info'},
  'blockchain_info_update': {'miner': 'OzCoin',
   'matches': [{'cb_match': 'ozco.in'},
    {'addr_match': '1GG9HQZchCRxPSBV5SwZ9GoYEVq9vVLGqU'}],
   'src': 'blockchain.info'}},
 'hash': '0000

In [77]:
blocks["159846"]

{'time': 1325270885,
 'cb': '70736a04ba760e1a040c9b0100522cfabe6d6dd5198c9a7f3796c9599592dc199792cc88c0a5588c2b9f5b491c1d8cbed5700601000000000000006f7a636f2e696eac1eeeed88',
 'addresses': ['1GG9HQZchCRxPSBV5SwZ9GoYEVq9vVLGqU'],
 'miner': '',
 'conflicts': 0,
 'attribution': '',
 'attributions': {'blockchain_info_marker': {'miner': 'OzCoin',
   'matches': [{'cb_match': 'ozco.in'}],
   'src': 'blockchain.info'},
  'blockchain_info': {'miner': 'OzCoin',
   'matches': [{'cb_match': 'ozco.in'}],
   'src': 'blockchain.info'},
  'blockchain_info_marker_update': {'miner': 'OzCoin',
   'matches': [{'cb_match': 'ozco.in'}],
   'src': 'blockchain.info'},
  'blockchain_info_update': {'miner': 'OzCoin',
   'matches': [{'cb_match': 'ozco.in'},
    {'addr_match': '1GG9HQZchCRxPSBV5SwZ9GoYEVq9vVLGqU'}],
   'src': 'blockchain.info'},
  'blockchain_info_address_update': {'miner': 'OzCoin',
   'matches': [{'addr_match': '1GG9HQZchCRxPSBV5SwZ9GoYEVq9vVLGqU'}],
   'src': 'blockchain.info'}},
 'hash': '0000

In [78]:
blocks["159964"]

{'time': 1325333051,
 'cb': '70736a04ba760e1a04f9850000522cfabe6d6daa7399390129c5b877981adab8face0bd391539f3127cad753419a506405ced001000000000000006f7a636f2e696eac1eeeed88',
 'addresses': ['1GG9HQZchCRxPSBV5SwZ9GoYEVq9vVLGqU'],
 'miner': '',
 'conflicts': 0,
 'attribution': '',
 'attributions': {'blockchain_info_marker': {'miner': 'OzCoin',
   'matches': [{'cb_match': 'ozco.in'}],
   'src': 'blockchain.info'},
  'blockchain_info': {'miner': 'OzCoin',
   'matches': [{'cb_match': 'ozco.in'}],
   'src': 'blockchain.info'},
  'blockchain_info_address_update': {'miner': 'OzCoin',
   'matches': [{'addr_match': '1GG9HQZchCRxPSBV5SwZ9GoYEVq9vVLGqU'}],
   'src': 'blockchain.info'},
  'blockchain_info_marker_update': {'miner': 'OzCoin',
   'matches': [{'cb_match': 'ozco.in'}],
   'src': 'blockchain.info'},
  'blockchain_info_update': {'miner': 'OzCoin',
   'matches': [{'cb_match': 'ozco.in'},
    {'addr_match': '1GG9HQZchCRxPSBV5SwZ9GoYEVq9vVLGqU'}],
   'src': 'blockchain.info'}},
 'hash': '0000

Print and check random block:

In [79]:
util.get_sample(blocks)

{'time': 1430016717,
 'cb': '03cc65051d4d696e656420627920416e74506f6f6c20626a35268cc47e20553c52cd902900003e750700',
 'addresses': ['1Ebb8NfVmKMoGuMJCAEbVMv2dX8GnzgxSa'],
 'miner': '',
 'conflicts': 0,
 'attribution': '',
 'attributions': {'blockchain_info_marker': {'miner': 'AntPool',
   'matches': [{'cb_match': 'Mined by AntPool'}],
   'src': 'blockchain.info'},
  'blockchain_info': {'miner': 'AntPool',
   'matches': [{'cb_match': 'Mined by AntPool'}],
   'src': 'blockchain.info'},
  'blockchain_info_address_update': {'miner': 'AntPool',
   'matches': [{'addr_match': '1Ebb8NfVmKMoGuMJCAEbVMv2dX8GnzgxSa'}],
   'src': 'blockchain.info'},
  'blockchain_info_marker_update': {'miner': 'AntPool',
   'matches': [{'cb_match': 'Mined by AntPool'}],
   'src': 'blockchain.info'},
  'blockchain_info_update': {'miner': 'AntPool',
   'matches': [{'cb_match': 'Mined by AntPool'},
    {'addr_match': '1Ebb8NfVmKMoGuMJCAEbVMv2dX8GnzgxSa'}],
   'src': 'blockchain.info'}},
 'hash': '000000000000000004abe

Check how many addresses have been added by marker detection:

In [80]:
def check_for_miner_addresses_from_markers(miners):
    # check for miners addresses added based on markers 
    i = 0 
    for miner in miners:
        for addr in miners[ miner ][ util.D_ADDRESSES ]:
            if util.DD_SOURCES in miners[ miner ][ util.D_ADDRESSES ][ addr ]:
                if "cb marker" in miners[ miner ][ util.D_ADDRESSES ][ addr ][ util.DD_SOURCES ]:
                    #print("miner: ",miner," addr: ", addr)
                    i += 1
            #else:
                #print(miners[ miner ])
                #break
    return i
            

In [81]:
print("Added addresses by marker: ",util.check_for_miner_addresses_from_markers(miners_initial_blockchaininfo)) # 6021

Added addresses by marker:  6021


In [82]:
assert util.check_for_miner_addresses_from_markers(miners_initial_blockchaininfo) == 6021

Check if these addtions have caused address collisions:

In [83]:
def check_for_obvious_address_collisions(miners):
    # check for obvious address collisions:
    addr_collision = False
    address_conflicts = Counter()

    for m in miners:
        for a in miners[ m ][ util.D_ADDRESSES ].keys():
            address_conflicts[ a ] += 1

    for at in address_conflicts.most_common():
        if at[1] > 1:
            #address_conflicts_list.append( at[0] )
            print(at[0],":",at[1])
            addr_collision = True
    return addr_collision

In [84]:
assert util.check_for_obvious_address_collisions(miners_initial_blockchaininfo) == False

In [85]:
util.get_sample(miners_initial_blockchaininfo)

{'names': {'Bravo Mining': {'url': 'http://www.bravo-mining.com',
   'currencies': ['BTC'],
   'fullName': '',
   'firstUsed': 0,
   'lastUsed': 0,
   'sources': ['blockchain.info github']}},
 'markers': {'/bravo-mining/': {'currencies': ['BTC'],
   'firstUsed': 0,
   'lastUsed': 0,
   'sources': ['blockchain.info github']}},
 'addresses': {'14KxdJ7DfAPsxCYkpcDTcKotjeProcfRPo': {'sources': ['cb marker']}}}

## Fresh btccom only attribution and create miners_initial_btccom.json

Attribute all blocks based soly on the btccom attribution

In [86]:
# Boostrap pools files with blockchain.info github data
# https://github.com/blockchain/Blockchain-Known-Pools/tree/82ed31956388e3950845cc2faeaf6679a057ee5b
miners_initial_btccom = dict()
miners_initial_btccom.clear()

miners_initial_btccom = add_makers_from_source(pools_btccom,miners_initial_btccom,source=["btccom github",])

Multiple markers for pool: OzCoin
Multiple markers for pool: TripleMining
Multiple markers for pool: Polmine
Multiple markers for pool: AntPool
Multiple markers for pool: AntPool
Multiple markers for pool: BTCC
Multiple markers for pool: BTCC
Multiple markers for pool: BTCC
Multiple markers for pool: BWPool
Multiple markers for pool: BitFury
Multiple markers for pool: ViaBTC
Multiple markers for pool: CanoePool
Multiple markers for pool: PHash.IO


In [87]:
miners_initial_btccom = add_addresses_from_source(pools_btccom,miners_initial_btccom,source=["btccom github",])

CONFLICTING URL: 'https://www.bitfarms.io/'
CONFLICTING URL: 'https://www.bitarms.io/'
new pool: BTC Nuggets
new pool: Huobi
new pool: CloudHashing
new pool: digitalX Mintsy
new pool: Telco 214
new pool: BTC Pool Party
new pool: Multipool
new pool: transactioncoinmining
new pool: BTCDig
new pool: Tricky's BTC Pool
new pool: BTCMP
new pool: Eobot
new pool: UNOMP
new pool: Patels
new pool: GoGreenLight
new pool: BitcoinIndia
CONFLICTING URL: 'https://btc.canoepool.com/'
CONFLICTING URL: 'https://www.canoepool.com/'
new pool: EkanemBTC
new pool: CANOE
new pool: tiger


In [88]:
# rename miners according to blockchain.info
# Therefore we create a new empty miner and move the existing one into it
# 'BTCC' => 'BTCC Pool'
util.add_miner('BTCC Pool',miners_initial_btccom)
util.unify_miners('BTCC','BTCC Pool',miners_initial_btccom)
print()

In [89]:
# 'BWPool' => 'BW.COM'
util.add_miner('BW.COM',miners_initial_btccom)
util.unify_miners('BWPool','BW.COM',miners_initial_btccom)
print()


In [90]:
# 'Solo CK' => 'Solo CKPool'
util.add_miner('Solo CKPool',miners_initial_btccom)
util.unify_miners('Solo CK','Solo CKPool',miners_initial_btccom)
print()

In [91]:
# 'BitClub' => 'BitClub Network'
util.add_miner('BitClub Network',miners_initial_btccom)
util.unify_miners('BitClub','BitClub Network',miners_initial_btccom)
print()

In [92]:
# 'Huobi.pool' => 'Huobi'
#util.add_miner('Huobi',miners_initial_btccom) # already exists 
util.unify_miners('Huobi.pool','Huobi',miners_initial_btccom)
print()

In [93]:
# 'sigmapool.com' => 'SigmaPool.com'
util.add_miner('SigmaPool.com',miners_initial_btccom)  
util.unify_miners('sigmapool.com','SigmaPool.com',miners_initial_btccom)
print()

In [94]:
# 'EXX&BW' => 'xbtc.exx.com&bw.com'
util.add_miner('xbtc.exx.com&bw.com',miners_initial_btccom) 
util.unify_miners('EXX&BW','xbtc.exx.com&bw.com',miners_initial_btccom)
print()

In [95]:
# 'haominer' => 'Haominer'
util.add_miner('Haominer',miners_initial_btccom) # already exists 
util.unify_miners('haominer','Haominer',miners_initial_btccom)
print()

In [96]:
# "Patels" => "Patel's Mining pool"
util.add_miner("Patel's Mining pool",miners_initial_btccom) # already exists 
util.unify_miners("Patels","Patel's Mining pool",miners_initial_btccom)
print()

In [97]:
# 'digitalX Mintsy' => 'digitalBTC'
util.add_miner('digitalBTC',miners_initial_btccom) # already exists 
util.unify_miners('digitalX Mintsy','digitalBTC',miners_initial_btccom)
print()

In [98]:
# 'NiceHash' => 'NiceHash Solo'
util.add_miner('NiceHash Solo',miners_initial_btccom) # already exists 
util.unify_miners('NiceHash','NiceHash Solo',miners_initial_btccom)
print()

In [99]:
miners_initial_btccom["DCExploration"]

{'names': {'DCExploration': {'url': 'http://dcexploration.cn',
   'currencies': ['BTC'],
   'fullName': '',
   'firstUsed': 0,
   'lastUsed': 0,
   'sources': ['btccom github']}},
 'markers': {'/DCExploration/': {'currencies': ['BTC'],
   'firstUsed': 0,
   'lastUsed': 0,
   'sources': ['btccom github']}},
 'addresses': {}}

In [100]:
miners_initial_btccom["DCEX"]

{'names': {'DCEX': {'url': 'http://dcexploration.cn',
   'currencies': ['BTC'],
   'fullName': '',
   'firstUsed': 0,
   'lastUsed': 0,
   'sources': ['btccom github']}},
 'markers': {'/DCEX/': {'currencies': ['BTC'],
   'firstUsed': 0,
   'lastUsed': 0,
   'sources': ['btccom github']}},
 'addresses': {}}

In [101]:
util.unify_miners("DCEX","DCExploration",miners_initial_btccom)
print()

In [102]:
miners_initial_btccom["Bitcoin India"]

{'names': {'Bitcoin India': {'url': 'https://bitcoin-india.org',
   'currencies': ['BTC'],
   'fullName': '',
   'firstUsed': 0,
   'lastUsed': 0,
   'sources': ['btccom github']}},
 'markers': {'/Bitcoin-India/': {'currencies': ['BTC'],
   'firstUsed': 0,
   'lastUsed': 0,
   'sources': ['btccom github']}},
 'addresses': {}}

In [103]:
miners_initial_btccom["BitcoinIndia"]

{'names': {'BitcoinIndia': {'url': 'https://pool.bitcoin-india.org/',
   'currencies': ['BTC'],
   'fullName': '',
   'firstUsed': 0,
   'lastUsed': 0,
   'sources': ['btccom github']}},
 'markers': {},
 'addresses': {'1AZ6BkCo4zgTuuLpRStJH8iNsehXTMp456': {'currencies': ['BTC'],
   'firstUsed': 0,
   'lastUsed': 0,
   'sources': ['btccom github']}}}

In [104]:
util.unify_miners("BitcoinIndia","Bitcoin India",miners_initial_btccom)
print()

In [105]:
miners_initial_btccom["CANOE"]

{'names': {'CANOE': {'url': 'https://www.canoepool.com',
   'currencies': ['BTC'],
   'fullName': '',
   'firstUsed': 0,
   'lastUsed': 0,
   'sources': ['btccom github']}},
 'markers': {},
 'addresses': {'1Afcpc2FpPnREU6i52K3cicmHdvYRAH9Wo': {'currencies': ['BTC'],
   'firstUsed': 0,
   'lastUsed': 0,
   'sources': ['btccom github']}}}

In [106]:
miners_initial_btccom["CanoePool"]

{'names': {'CanoePool': {'url': 'https://www.canoepool.com/',
   'currencies': ['BTC'],
   'fullName': '',
   'firstUsed': 0,
   'lastUsed': 0,
   'sources': ['btccom github']}},
 'markers': {'/canoepool/': {'currencies': ['BTC'],
   'firstUsed': 0,
   'lastUsed': 0,
   'sources': ['btccom github']},
  '/CANOE/': {'currencies': ['BTC'],
   'firstUsed': 0,
   'lastUsed': 0,
   'sources': ['btccom github']}},
 'addresses': {'1GP8eWArgpwRum76saJS4cZKCHWJHs9PQo': {'currencies': ['BTC'],
   'firstUsed': 0,
   'lastUsed': 0,
   'sources': ['btccom github']}}}

In [107]:
util.unify_miners("CanoePool","CANOE",miners_initial_btccom)
print()

In [108]:
util.get_sample(miners_initial_btccom)

{'names': {'Nexious': {'url': 'https://nexious.com/',
   'currencies': ['BTC'],
   'fullName': '',
   'firstUsed': 0,
   'lastUsed': 0,
   'sources': ['btccom github']}},
 'markers': {'/Nexious/': {'currencies': ['BTC'],
   'firstUsed': 0,
   'lastUsed': 0,
   'sources': ['btccom github']}},
 'addresses': {'1GBo1f2tzVx5jScV9kJXPUP9RjvYXuNzV7': {'currencies': ['BTC'],
   'firstUsed': 0,
   'lastUsed': 0,
   'sources': ['btccom github']}}}

In [109]:
with open(miners_initial_btccom_json_file, 'w') as outfile:
    json.dump(miners_initial_btccom, outfile)

In [110]:
# this dict will be updated when coinbase matches are found and there is only one address 
# as the coinbase output
miners = dict()
miners.clear()
miners = copy.deepcopy(miners_initial_btccom)

In [111]:
# attribute blocks to miners according to btccom initial mapping
(blocks,miners_initial_btccom,conflicts) = util.attribute_blocks(blocks=blocks,
                                   miners_dict=miners_initial_btccom,
                                   addr_attr=util.DD_BTCCOM_ADDR_ATTR,
                                   marker_attr=util.DD_BTCCOM_MARK_ATTR,
                                   both_attr=util.DD_BTCCOM_ATTR,
                                   source="btccom",
                                   override=True)

482000
Message    =  Addr and Cb match differ
Blockheight=  482059
Miner1     =  Waterhole
Miner2     =  BTC.com
Coinbase   =  030b5b0704465fa1592f4254432e434f4d2ffabe6d6dff48161efa0f0b44fc9463e380e6b585ff91ac3f9c973d156dfff35fd553cdb101000000000000000228124b7faf010000000000
CoinbaseStr=  b'\x03\x0b[\x07\x04F_\xa1Y/BTC.COM/\xfa\xbemm\xffH\x16\x1e\xfa\x0f\x0bD\xfc\x94c\xe3\x80\xe6\xb5\x85\xff\x91\xac?\x9c\x97=\x15m\xff\xf3_\xd5S\xcd\xb1\x01\x00\x00\x00\x00\x00\x00\x00\x02(\x12K\x7f\xaf\x01\x00\x00\x00\x00\x00'
Addesses   =  1FLH1SoLv4U68yUERhDiWzrJn5TggMqkaZ
addr_match =  [('Waterhole', {'addr_match': '1FLH1SoLv4U68yUERhDiWzrJn5TggMqkaZ'})]
cb_match   =  [('BTC.com', {'cb_match': '/BTC.COM/'})]

Message    =  Addr and Cb match differ
Blockheight=  482221
Miner1     =  Waterhole
Miner2     =  BTC.com
Coinbase   =  03ad5b0704a4e6a2592f4254432e434f4d2ffabe6d6d0dd4f6c7dac894c84821c9fed8c15a621583ada8e4177c182c011cab52e2d6d0010000000000000001292f8e92ae010000000000
CoinbaseStr=  b'\x03\xad[\x

In [112]:
len(conflicts) # 3

3

In [113]:
assert len(conflicts) == 3

In [114]:
blocks["524045"]

{'time': 1527103163,
 'cb': '030dff0704bbbe055b08fabe6d6d5161e32de4ba2440240366a8777307840c73e448a8e5239e663916c17e0be144010000000000000078054a364b5fe107182f426974636f696e2d556b7261696e652e636f6d2e75612f',
 'addresses': ['165GCEAx81wce33FWEnPCRhdjcXCrBJdKn'],
 'miner': '',
 'conflicts': 0,
 'attribution': '',
 'attributions': {'blockchain_info_address_update': {'miner': 'BitcoinRussia',
   'matches': [{'addr_match': '165GCEAx81wce33FWEnPCRhdjcXCrBJdKn'}],
   'src': 'blockchain.info'},
  'blockchain_info_update': {'miner': 'BitcoinRussia',
   'matches': [{'addr_match': '165GCEAx81wce33FWEnPCRhdjcXCrBJdKn'}],
   'src': 'blockchain.info'},
  'btccom_address': {'miner': 'BitcoinRussia',
   'matches': [{'addr_match': '165GCEAx81wce33FWEnPCRhdjcXCrBJdKn'}],
   'src': 'btccom'},
  'btccom_marker': {'miner': 'Bitcoin-Ukraine',
   'matches': [{'cb_match': '/Bitcoin-Ukraine.com.ua/'}],
   'src': 'btccom'}},
 'hash': '00000000000000000038b67cc69238ebc13b7c63410200d03b67be05ed8af0e0',
 'phash': No

In [115]:
# attribute blocks to miners according to btccom initial mapping
# update initial mapping when coinbase marker matches and there is only one output address of coinbase tx
(blocks,miners_initial_btccom,conflicts) = util.attribute_blocks(blocks=blocks,
                                   miners_dict=miners_initial_btccom,
                                   addr_attr=util.DD_BTCCOM_ADDR_ATTR + "_update",
                                   marker_attr=util.DD_BTCCOM_MARK_ATTR + "_update",
                                   both_attr=util.DD_BTCCOM_ATTR + "_update",
                                   source="btccom",
                                   override=True,
                                   update=True)

160000
Message    =  Addr and Cb match differ
Blockheight=  160367
Miner1     =  OzCoin
Miner2     =  Yourbtc.net
Coinbase   =  70736a04ba760e1a046c730300522cfabe6d6dad6ee140dbdc015c8da85b8e3b636dc896e7df9203a7cd2c41e858503563ac130100000000000000796f75726274632e6e6574ac1eeeed88
CoinbaseStr=  b'psj\x04\xbav\x0e\x1a\x04ls\x03\x00R,\xfa\xbemm\xadn\xe1@\xdb\xdc\x01\\\x8d\xa8[\x8e;cm\xc8\x96\xe7\xdf\x92\x03\xa7\xcd,A\xe8XP5c\xac\x13\x01\x00\x00\x00\x00\x00\x00\x00yourbtc.net\xac\x1e\xee\xed\x88'
Addesses   =  1GG9HQZchCRxPSBV5SwZ9GoYEVq9vVLGqU
addr_match =  [('OzCoin', {'addr_match': '1GG9HQZchCRxPSBV5SwZ9GoYEVq9vVLGqU'})]
cb_match   =  [('Yourbtc.net', {'cb_match': 'yourbtc.net'})]

Message    =  Addr and Cb match differ
Blockheight=  160464
Miner1     =  OzCoin
Miner2     =  Yourbtc.net
Coinbase   =  70736a04ba760e1a04136c1100522cfabe6d6d7d24b129b8b97a267e531f9d816c6f435c9bb8a5fcc78b83360a5fee452c9e3e0100000000000000796f75726274632e6e6574ac1eeeed88
CoinbaseStr=  b'psj\x04\xbav\x0e\x1a\x04


Message    =  Addr and Cb match differ
Blockheight=  161776
Miner1     =  OzCoin
Miner2     =  Yourbtc.net
Coinbase   =  70736a04d7690d1a04fbb71201522cfabe6d6d87c57bc803e3d93d83bc53bfedbd6009a76e1a4006c936dc8b18d89fcdc8f39e0100000000000000796f75726274632e6e6574ac1eeeed88
CoinbaseStr=  b'psj\x04\xd7i\r\x1a\x04\xfb\xb7\x12\x01R,\xfa\xbemm\x87\xc5{\xc8\x03\xe3\xd9=\x83\xbcS\xbf\xed\xbd`\t\xa7n\x1a@\x06\xc96\xdc\x8b\x18\xd8\x9f\xcd\xc8\xf3\x9e\x01\x00\x00\x00\x00\x00\x00\x00yourbtc.net\xac\x1e\xee\xed\x88'
Addesses   =  1GG9HQZchCRxPSBV5SwZ9GoYEVq9vVLGqU
addr_match =  [('OzCoin', {'addr_match': '1GG9HQZchCRxPSBV5SwZ9GoYEVq9vVLGqU'})]
cb_match   =  [('Yourbtc.net', {'cb_match': 'yourbtc.net'})]

Message    =  Addr and Cb match differ
Blockheight=  161796
Miner1     =  OzCoin
Miner2     =  Yourbtc.net
Coinbase   =  70736a04d7690d1a0469a8a203522cfabe6d6d31d5772c1102469d5a07818eba4f1813ee994c70f329852b3728370a9c29df280100000000000000796f75726274632e6e6574ac1eeeed88
CoinbaseStr=  b'psj\x04\xd7

Message    =  Addr and Cb match differ
Blockheight=  162205
Miner1     =  OzCoin
Miner2     =  Yourbtc.net
Coinbase   =  70736a04d7690d1a045ef95006522cfabe6d6da36c2d9a23155dd5426177ac77a0e4e218f7fb5ff9d6dc8e9ae6b19c4eb574a40100000000000000796f75726274632e6e6574ac1eeeed88
CoinbaseStr=  b'psj\x04\xd7i\r\x1a\x04^\xf9P\x06R,\xfa\xbemm\xa3l-\x9a#\x15]\xd5Baw\xacw\xa0\xe4\xe2\x18\xf7\xfb_\xf9\xd6\xdc\x8e\x9a\xe6\xb1\x9cN\xb5t\xa4\x01\x00\x00\x00\x00\x00\x00\x00yourbtc.net\xac\x1e\xee\xed\x88'
Addesses   =  1GG9HQZchCRxPSBV5SwZ9GoYEVq9vVLGqU
addr_match =  [('OzCoin', {'addr_match': '1GG9HQZchCRxPSBV5SwZ9GoYEVq9vVLGqU'})]
cb_match   =  [('Yourbtc.net', {'cb_match': 'yourbtc.net'})]

Message    =  Addr and Cb match differ
Blockheight=  162221
Miner1     =  OzCoin
Miner2     =  Yourbtc.net
Coinbase   =  70736a04d7690d1a0429c62b00522cfabe6d6d6aa88a20b82c6e273be1575f5ac763eaea97d578ee8a0b5802da191932b98bc40100000000000000796f75726274632e6e6574ac1eeeed88
CoinbaseStr=  b"psj\x04\xd7i\r\x1a\x04)\xc6+

cb_match   =  [('Yourbtc.net', {'cb_match': 'yourbtc.net'})]

Message    =  Addr and Cb match differ
Blockheight=  162581
Miner1     =  OzCoin
Miner2     =  Yourbtc.net
Coinbase   =  70736a04d7690d1a0446686e00522cfabe6d6d6f791565b5fd4014b182d45b47d738d79c272d13834529596cb757c8c39724980100000000000000796f75726274632e6e6574ac1eeeed88
CoinbaseStr=  b"psj\x04\xd7i\r\x1a\x04Fhn\x00R,\xfa\xbemmoy\x15e\xb5\xfd@\x14\xb1\x82\xd4[G\xd78\xd7\x9c'-\x13\x83E)Yl\xb7W\xc8\xc3\x97$\x98\x01\x00\x00\x00\x00\x00\x00\x00yourbtc.net\xac\x1e\xee\xed\x88"
Addesses   =  1GG9HQZchCRxPSBV5SwZ9GoYEVq9vVLGqU
addr_match =  [('OzCoin', {'addr_match': '1GG9HQZchCRxPSBV5SwZ9GoYEVq9vVLGqU'})]
cb_match   =  [('Yourbtc.net', {'cb_match': 'yourbtc.net'})]

Message    =  Addr and Cb match differ
Blockheight=  162686
Miner1     =  OzCoin
Miner2     =  Yourbtc.net
Coinbase   =  70736a04d7690d1a04c14d3400522cfabe6d6ddc89b636b2eaf2e2990999df789fec6459b8f505c511ed7e07a6e73661601c810100000000000000796f75726274632e6e6574ac1eeeed


Message    =  Addr and Cb match differ
Blockheight=  163201
Miner1     =  OzCoin
Miner2     =  Yourbtc.net
Coinbase   =  70736a04d7690d1a04a38c2800522cfabe6d6d21ecaded32c33c2fe3bc13ae457d515e37f42964dc3d4151b6b620ee8e2f71be0100000000000000796f75726274632e6e6574ac1eeeed88
CoinbaseStr=  b'psj\x04\xd7i\r\x1a\x04\xa3\x8c(\x00R,\xfa\xbemm!\xec\xad\xed2\xc3</\xe3\xbc\x13\xaeE}Q^7\xf4)d\xdc=AQ\xb6\xb6 \xee\x8e/q\xbe\x01\x00\x00\x00\x00\x00\x00\x00yourbtc.net\xac\x1e\xee\xed\x88'
Addesses   =  1GG9HQZchCRxPSBV5SwZ9GoYEVq9vVLGqU
addr_match =  [('OzCoin', {'addr_match': '1GG9HQZchCRxPSBV5SwZ9GoYEVq9vVLGqU'})]
cb_match   =  [('Yourbtc.net', {'cb_match': 'yourbtc.net'})]

Message    =  Addr and Cb match differ
Blockheight=  163213
Miner1     =  OzCoin
Miner2     =  Yourbtc.net
Coinbase   =  70736a04d7690d1a048b653400522cfabe6d6dff7c38cf70c1509f64529e83a2011adba8b92ce74493d7eb312f28beacdd7ee10100000000000000796f75726274632e6e6574ac1eeeed88
CoinbaseStr=  b'psj\x04\xd7i\r\x1a\x04\x8be4\x00R,\xfa\xbe

...


Message    =  Addr and Cb match differ
Blockheight=  519591
Miner1     =  BTC.TOP
Miner2     =  CANOE
Coinbase   =  03a7ed0741d6b77d75d6ba4541d6b77d75c8eaea2f63616e6f65706f6f6c2fb700db5b0dc3000000000000
CoinbaseStr=  b'\x03\xa7\xed\x07A\xd6\xb7}u\xd6\xbaEA\xd6\xb7}u\xc8\xea\xea/canoepool/\xb7\x00\xdb[\r\xc3\x00\x00\x00\x00\x00\x00'
Addesses   =  147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq
addr_match =  [('BTC.TOP', {'addr_match': '147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq'})]
cb_match   =  [('CANOE', {'cb_match': '/canoepool/'})]

Message    =  Addr and Cb match differ
Blockheight=  519609
Miner1     =  BTC.TOP
Miner2     =  CANOE
Coinbase   =  03b9ed0741d6b786caddcd7e41d6b786c9d765a12f63616e6f65706f6f6c2fba005f42b556000000000000
CoinbaseStr=  b'\x03\xb9\xed\x07A\xd6\xb7\x86\xca\xdd\xcd~A\xd6\xb7\x86\xc9\xd7e\xa1/canoepool/\xba\x00_B\xb5V\x00\x00\x00\x00\x00\x00'
Addesses   =  147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq
addr_match =  [('BTC.TOP', {'addr_match': '147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq'})]
cb_match


Message    =  Addr and Cb match differ
Blockheight=  520825
Miner1     =  BTC.TOP
Miner2     =  CANOE
Coinbase   =  0379f20741d6ba4aa28cddb941d6ba4aa17c8f4f2f63616e6f65706f6f6c2fba005414ddc3010000000000
CoinbaseStr=  b'\x03y\xf2\x07A\xd6\xbaJ\xa2\x8c\xdd\xb9A\xd6\xbaJ\xa1|\x8fO/canoepool/\xba\x00T\x14\xdd\xc3\x01\x00\x00\x00\x00\x00'
Addesses   =  147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq
addr_match =  [('BTC.TOP', {'addr_match': '147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq'})]
cb_match   =  [('CANOE', {'cb_match': '/canoepool/'})]
521000
Message    =  Addr and Cb match differ
Blockheight=  521032
Miner1     =  BTC.TOP
Miner2     =  CANOE
Coinbase   =  0348f30741d6bac12359b18641d6bac122cfe4062f63616e6f65706f6f6c2fb70052c57a70010000000000
CoinbaseStr=  b'\x03H\xf3\x07A\xd6\xba\xc1#Y\xb1\x86A\xd6\xba\xc1"\xcf\xe4\x06/canoepool/\xb7\x00R\xc5zp\x01\x00\x00\x00\x00\x00'
Addesses   =  147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq
addr_match =  [('BTC.TOP', {'addr_match': '147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq'})]
cb_mat

523000
Message    =  Addr and Cb match differ
Blockheight=  523113
Miner1     =  BTC.TOP
Miner2     =  CANOE
Coinbase   =  0369fb0741d6bf5c1632db1441d6bf5c155bd9972f63616e6f65706f6f6c2fba0002024303004000000000
CoinbaseStr=  b'\x03i\xfb\x07A\xd6\xbf\\\x162\xdb\x14A\xd6\xbf\\\x15[\xd9\x97/canoepool/\xba\x00\x02\x02C\x03\x00@\x00\x00\x00\x00'
Addesses   =  147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq
addr_match =  [('BTC.TOP', {'addr_match': '147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq'})]
cb_match   =  [('CANOE', {'cb_match': '/canoepool/'})]

Message    =  Addr and Cb match differ
Blockheight=  523115
Miner1     =  BTC.TOP
Miner2     =  CANOE
Coinbase   =  036bfb0741d6bf5d32ffb0fa41d6bf5d325d22f92f63616e6f65706f6f6c2fb7004ce4bb6a000000000000
CoinbaseStr=  b'\x03k\xfb\x07A\xd6\xbf]2\xff\xb0\xfaA\xd6\xbf]2]"\xf9/canoepool/\xb7\x00L\xe4\xbbj\x00\x00\x00\x00\x00\x00'
Addesses   =  147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq
addr_match =  [('BTC.TOP', {'addr_match': '147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq'})]
cb_match   = 


Message    =  Addr and Cb match differ
Blockheight=  525559
Miner1     =  BTC.TOP
Miner2     =  CANOE
Coinbase   =  03f7040841d6c4827eb5b6a141d6c4827e7b72da2f63616e6f65706f6f6c2fb70037c928a2010000000000
CoinbaseStr=  b'\x03\xf7\x04\x08A\xd6\xc4\x82~\xb5\xb6\xa1A\xd6\xc4\x82~{r\xda/canoepool/\xb7\x007\xc9(\xa2\x01\x00\x00\x00\x00\x00'
Addesses   =  147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq
addr_match =  [('BTC.TOP', {'addr_match': '147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq'})]
cb_match   =  [('CANOE', {'cb_match': '/canoepool/'})]

Message    =  Addr and Cb match differ
Blockheight=  525566
Miner1     =  BTC.TOP
Miner2     =  CANOE
Coinbase   =  03fe040841d6c4863752a84241d6c48636fbf1fa2f63616e6f65706f6f6c2fb700438325c8010000000000
CoinbaseStr=  b'\x03\xfe\x04\x08A\xd6\xc4\x867R\xa8BA\xd6\xc4\x866\xfb\xf1\xfa/canoepool/\xb7\x00C\x83%\xc8\x01\x00\x00\x00\x00\x00'
Addesses   =  147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq
addr_match =  [('BTC.TOP', {'addr_match': '147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq'})]
cb_match 

Blockheight=  528216
Miner1     =  BTC.TOP
Miner2     =  CANOE
Coinbase   =  03580f0841d6ca423ff5cdc041d6ca423eff18902f63616e6f65706f6f6c2fba0047b68fcb010000000000
CoinbaseStr=  b'\x03X\x0f\x08A\xd6\xcaB?\xf5\xcd\xc0A\xd6\xcaB>\xff\x18\x90/canoepool/\xba\x00G\xb6\x8f\xcb\x01\x00\x00\x00\x00\x00'
Addesses   =  147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq
addr_match =  [('BTC.TOP', {'addr_match': '147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq'})]
cb_match   =  [('CANOE', {'cb_match': '/canoepool/'})]

Message    =  Addr and Cb match differ
Blockheight=  528771
Miner1     =  BTC.TOP
Miner2     =  CANOE
Coinbase   =  0383110841d6cb67c64aebcf41d6cb67c5af8d622f63616e6f65706f6f6c2fba004c772248010000000000
CoinbaseStr=  b'\x03\x83\x11\x08A\xd6\xcbg\xc6J\xeb\xcfA\xd6\xcbg\xc5\xaf\x8db/canoepool/\xba\x00Lw"H\x01\x00\x00\x00\x00\x00'
Addesses   =  147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq
addr_match =  [('BTC.TOP', {'addr_match': '147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq'})]
cb_match   =  [('CANOE', {'cb_match': '/canoepool/'})]


In [116]:
len(conflicts)

388

In [117]:
i = 0 
for c in conflicts:
    if c[ "miner1" ] == "CANOE" or c[ "miner2"] == "CANOE":
        i += 1
i

322

In [118]:
# attribute blocks to miners according to btccom initial mapping 
# updated with single addresses on which coinbase marker matched 
(blocks,miners_initial_btccom,conflicts) = util.attribute_blocks(blocks=blocks,
                                   miners_dict=miners_initial_btccom,
                                   addr_attr=util.DD_BTCCOM_ADDR_ATTR + "_update",
                                   marker_attr=util.DD_BTCCOM_MARK_ATTR + "_update",
                                   both_attr=util.DD_BTCCOM_ATTR + "_update",
                                   source="btccom",
                                   override=True,
                                   update=True)

160000
Message    =  Addr and Cb match differ
Blockheight=  160367
Miner1     =  OzCoin
Miner2     =  Yourbtc.net
Coinbase   =  70736a04ba760e1a046c730300522cfabe6d6dad6ee140dbdc015c8da85b8e3b636dc896e7df9203a7cd2c41e858503563ac130100000000000000796f75726274632e6e6574ac1eeeed88
CoinbaseStr=  b'psj\x04\xbav\x0e\x1a\x04ls\x03\x00R,\xfa\xbemm\xadn\xe1@\xdb\xdc\x01\\\x8d\xa8[\x8e;cm\xc8\x96\xe7\xdf\x92\x03\xa7\xcd,A\xe8XP5c\xac\x13\x01\x00\x00\x00\x00\x00\x00\x00yourbtc.net\xac\x1e\xee\xed\x88'
Addesses   =  1GG9HQZchCRxPSBV5SwZ9GoYEVq9vVLGqU
addr_match =  [('OzCoin', {'addr_match': '1GG9HQZchCRxPSBV5SwZ9GoYEVq9vVLGqU'})]
cb_match   =  [('Yourbtc.net', {'cb_match': 'yourbtc.net'})]

Message    =  Addr and Cb match differ
Blockheight=  160464
Miner1     =  OzCoin
Miner2     =  Yourbtc.net
Coinbase   =  70736a04ba760e1a04136c1100522cfabe6d6d7d24b129b8b97a267e531f9d816c6f435c9bb8a5fcc78b83360a5fee452c9e3e0100000000000000796f75726274632e6e6574ac1eeeed88
CoinbaseStr=  b'psj\x04\xbav\x0e\x1a\x04

Message    =  Addr and Cb match differ
Blockheight=  161881
Miner1     =  OzCoin
Miner2     =  Yourbtc.net
Coinbase   =  70736a04d7690d1a040dbcaa01522cfabe6d6d3f8d5d406aacef2699b5b8001c12df9186b6b27846b53815b07a5370771874700100000000000000796f75726274632e6e6574ac1eeeed88
CoinbaseStr=  b'psj\x04\xd7i\r\x1a\x04\r\xbc\xaa\x01R,\xfa\xbemm?\x8d]@j\xac\xef&\x99\xb5\xb8\x00\x1c\x12\xdf\x91\x86\xb6\xb2xF\xb58\x15\xb0zSpw\x18tp\x01\x00\x00\x00\x00\x00\x00\x00yourbtc.net\xac\x1e\xee\xed\x88'
Addesses   =  1GG9HQZchCRxPSBV5SwZ9GoYEVq9vVLGqU
addr_match =  [('OzCoin', {'addr_match': '1GG9HQZchCRxPSBV5SwZ9GoYEVq9vVLGqU'})]
cb_match   =  [('Yourbtc.net', {'cb_match': 'yourbtc.net'})]

Message    =  Addr and Cb match differ
Blockheight=  161899
Miner1     =  OzCoin
Miner2     =  Yourbtc.net
Coinbase   =  70736a04d7690d1a042e70cc01522cfabe6d6d20dcbd6fec99828e60aea932c70bbf272de0ac34f88fd37885af139b2a1511f40100000000000000796f75726274632e6e6574ac1eeeed88
CoinbaseStr=  b"psj\x04\xd7i\r\x1a\x04.p\xcc\x01R


Message    =  Addr and Cb match differ
Blockheight=  162273
Miner1     =  OzCoin
Miner2     =  Yourbtc.net
Coinbase   =  70736a04d7690d1a0457274c00522cfabe6d6d7a0dd4dae39012aacf0fcd6247e472bd3ab673aa5b89cb13f648ccefb3d2adce0100000000000000796f75726274632e6e6574ac1eeeed88
CoinbaseStr=  b"psj\x04\xd7i\r\x1a\x04W'L\x00R,\xfa\xbemmz\r\xd4\xda\xe3\x90\x12\xaa\xcf\x0f\xcdbG\xe4r\xbd:\xb6s\xaa[\x89\xcb\x13\xf6H\xcc\xef\xb3\xd2\xad\xce\x01\x00\x00\x00\x00\x00\x00\x00yourbtc.net\xac\x1e\xee\xed\x88"
Addesses   =  1GG9HQZchCRxPSBV5SwZ9GoYEVq9vVLGqU
addr_match =  [('OzCoin', {'addr_match': '1GG9HQZchCRxPSBV5SwZ9GoYEVq9vVLGqU'})]
cb_match   =  [('Yourbtc.net', {'cb_match': 'yourbtc.net'})]

Message    =  Addr and Cb match differ
Blockheight=  162352
Miner1     =  OzCoin
Miner2     =  Yourbtc.net
Coinbase   =  70736a04d7690d1a0414613c00522cfabe6d6d6f55a559d7837637603cdda0b81b52962de5c7b2bddead1c95077cf14e6b14d40100000000000000796f75726274632e6e6574ac1eeeed88
CoinbaseStr=  b'psj\x04\xd7i\r\x1a\x04\

Message    =  Addr and Cb match differ
Blockheight=  162746
Miner1     =  OzCoin
Miner2     =  Yourbtc.net
Coinbase   =  70736a04d7690d1a04591f1400522cfabe6d6d0b1de5aab4e9f2be360cdd841c6c14e048c214d2c6c9e01e06e9a352502b48e70100000000000000796f75726274632e6e6574ac1eeeed88
CoinbaseStr=  b'psj\x04\xd7i\r\x1a\x04Y\x1f\x14\x00R,\xfa\xbemm\x0b\x1d\xe5\xaa\xb4\xe9\xf2\xbe6\x0c\xdd\x84\x1cl\x14\xe0H\xc2\x14\xd2\xc6\xc9\xe0\x1e\x06\xe9\xa3RP+H\xe7\x01\x00\x00\x00\x00\x00\x00\x00yourbtc.net\xac\x1e\xee\xed\x88'
Addesses   =  1GG9HQZchCRxPSBV5SwZ9GoYEVq9vVLGqU
addr_match =  [('OzCoin', {'addr_match': '1GG9HQZchCRxPSBV5SwZ9GoYEVq9vVLGqU'})]
cb_match   =  [('Yourbtc.net', {'cb_match': 'yourbtc.net'})]

Message    =  Addr and Cb match differ
Blockheight=  162778
Miner1     =  OzCoin
Miner2     =  Yourbtc.net
Coinbase   =  70736a04d7690d1a048b664e00522cfabe6d6d42179d6ed820e16af5461986c134cd013fd6991e5f0b9ff8d7b96ccc8aaa66300100000000000000796f75726274632e6e6574ac1eeeed88
CoinbaseStr=  b'psj\x04\xd7i\

482000
Message    =  Addr and Cb match differ
Blockheight=  482059
Miner1     =  Waterhole
Miner2     =  BTC.com
Coinbase   =  030b5b0704465fa1592f4254432e434f4d2ffabe6d6dff48161efa0f0b44fc9463e380e6b585ff91ac3f9c973d156dfff35fd553cdb101000000000000000228124b7faf010000000000
CoinbaseStr=  b'\x03\x0b[\x07\x04F_\xa1Y/BTC.COM/\xfa\xbemm\xffH\x16\x1e\xfa\x0f\x0bD\xfc\x94c\xe3\x80\xe6\xb5\x85\xff\x91\xac?\x9c\x97=\x15m\xff\xf3_\xd5S\xcd\xb1\x01\x00\x00\x00\x00\x00\x00\x00\x02(\x12K\x7f\xaf\x01\x00\x00\x00\x00\x00'
Addesses   =  1FLH1SoLv4U68yUERhDiWzrJn5TggMqkaZ
addr_match =  [('Waterhole', {'addr_match': '1FLH1SoLv4U68yUERhDiWzrJn5TggMqkaZ'})]
cb_match   =  [('BTC.com', {'cb_match': '/BTC.COM/'})]

Message    =  Addr and Cb match differ
Blockheight=  482221
Miner1     =  Waterhole
Miner2     =  BTC.com
Coinbase   =  03ad5b0704a4e6a2592f4254432e434f4d2ffabe6d6d0dd4f6c7dac894c84821c9fed8c15a621583ada8e4177c182c011cab52e2d6d0010000000000000001292f8e92ae010000000000
CoinbaseStr=  b'\x03\xad[\x

...

cb_match   =  [('CANOE', {'cb_match': '/canoepool/'})]

Message    =  Addr and Cb match differ
Blockheight=  519888
Miner1     =  BTC.TOP
Miner2     =  CANOE
Coinbase   =  03d0ee0741d6b82aa2de9e2d41d6b82aa1d46af52f63616e6f65706f6f6c2fba004bc451d5010000000000
CoinbaseStr=  b'\x03\xd0\xee\x07A\xd6\xb8*\xa2\xde\x9e-A\xd6\xb8*\xa1\xd4j\xf5/canoepool/\xba\x00K\xc4Q\xd5\x01\x00\x00\x00\x00\x00'
Addesses   =  147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq
addr_match =  [('BTC.TOP', {'addr_match': '147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq'})]
cb_match   =  [('CANOE', {'cb_match': '/canoepool/'})]

Message    =  Addr and Cb match differ
Blockheight=  519943
Miner1     =  BTC.TOP
Miner2     =  CANOE
Coinbase   =  0307ef0741d6b84e78f632d541d6b84e77e280f32f63616e6f65706f6f6c2fb700b38116a5000000000000
CoinbaseStr=  b'\x03\x07\xef\x07A\xd6\xb8Nx\xf62\xd5A\xd6\xb8Nw\xe2\x80\xf3/canoepool/\xb7\x00\xb3\x81\x16\xa5\x00\x00\x00\x00\x00\x00'
Addesses   =  147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq
addr_match =  [('BTC.TOP', {'addr

521000
Message    =  Addr and Cb match differ
Blockheight=  521032
Miner1     =  BTC.TOP
Miner2     =  CANOE
Coinbase   =  0348f30741d6bac12359b18641d6bac122cfe4062f63616e6f65706f6f6c2fb70052c57a70010000000000
CoinbaseStr=  b'\x03H\xf3\x07A\xd6\xba\xc1#Y\xb1\x86A\xd6\xba\xc1"\xcf\xe4\x06/canoepool/\xb7\x00R\xc5zp\x01\x00\x00\x00\x00\x00'
Addesses   =  147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq
addr_match =  [('BTC.TOP', {'addr_match': '147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq'})]
cb_match   =  [('CANOE', {'cb_match': '/canoepool/'})]

Message    =  Addr and Cb match differ
Blockheight=  521045
Miner1     =  BTC.TOP
Miner2     =  CANOE
Coinbase   =  0355f30741d6bac7fa4a70a741d6bac7fa1eea5c2f63616e6f65706f6f6c2fb70041f0fbb3010000000000
CoinbaseStr=  b'\x03U\xf3\x07A\xd6\xba\xc7\xfaJp\xa7A\xd6\xba\xc7\xfa\x1e\xea\\/canoepool/\xb7\x00A\xf0\xfb\xb3\x01\x00\x00\x00\x00\x00'
Addesses   =  147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq
addr_match =  [('BTC.TOP', {'addr_match': '147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq'})]
cb


Message    =  Addr and Cb match differ
Blockheight=  522710
Miner1     =  BTC.TOP
Miner2     =  CANOE
Coinbase   =  03d6f90741d6be88ea6fa32241d6be88e9ebb8462f63616e6f65706f6f6c2fb7002747e928000000000000
CoinbaseStr=  b'\x03\xd6\xf9\x07A\xd6\xbe\x88\xeao\xa3"A\xd6\xbe\x88\xe9\xeb\xb8F/canoepool/\xb7\x00\'G\xe9(\x00\x00\x00\x00\x00\x00'
Addesses   =  147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq
addr_match =  [('BTC.TOP', {'addr_match': '147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq'})]
cb_match   =  [('CANOE', {'cb_match': '/canoepool/'})]

Message    =  Addr and Cb match differ
Blockheight=  522814
Miner1     =  BTC.TOP
Miner2     =  CANOE
Coinbase   =  033efa0741d6bec2416125b241d6bec24138b84a2f63616e6f65706f6f6c2fba003be1ee24000000000000
CoinbaseStr=  b'\x03>\xfa\x07A\xd6\xbe\xc2Aa%\xb2A\xd6\xbe\xc2A8\xb8J/canoepool/\xba\x00;\xe1\xee$\x00\x00\x00\x00\x00\x00'
Addesses   =  147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq
addr_match =  [('BTC.TOP', {'addr_match': '147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq'})]
cb_match   =  [('

525000
Message    =  Addr and Cb match differ
Blockheight=  525172
Miner1     =  BTC.TOP
Miner2     =  CANOE
Coinbase   =  0374030841d6c3c3ac6624c241d6c3c3abfc36f12f63616e6f65706f6f6c2fb7004fa0f065000000000000
CoinbaseStr=  b'\x03t\x03\x08A\xd6\xc3\xc3\xacf$\xc2A\xd6\xc3\xc3\xab\xfc6\xf1/canoepool/\xb7\x00O\xa0\xf0e\x00\x00\x00\x00\x00\x00'
Addesses   =  147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq
addr_match =  [('BTC.TOP', {'addr_match': '147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq'})]
cb_match   =  [('CANOE', {'cb_match': '/canoepool/'})]

Message    =  Addr and Cb match differ
Blockheight=  525415
Miner1     =  BTC.TOP
Miner2     =  CANOE
Coinbase   =  0367040841d6c43c76284aa441d6c43c7513598a2f63616e6f65706f6f6c2fb70034501683010000000000
CoinbaseStr=  b'\x03g\x04\x08A\xd6\xc4<v(J\xa4A\xd6\xc4<u\x13Y\x8a/canoepool/\xb7\x004P\x16\x83\x01\x00\x00\x00\x00\x00'
Addesses   =  147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq
addr_match =  [('BTC.TOP', {'addr_match': '147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq'})]
cb_match   =  [

528000
Message    =  Addr and Cb match differ
Blockheight=  528047
Miner1     =  BTC.TOP
Miner2     =  CANOE
Coinbase   =  03af0e0841d6c9e2f9edf2a841d6c9e2f9597dfd2f63616e6f65706f6f6c2fba00366c0bce000000000000
CoinbaseStr=  b'\x03\xaf\x0e\x08A\xd6\xc9\xe2\xf9\xed\xf2\xa8A\xd6\xc9\xe2\xf9Y}\xfd/canoepool/\xba\x006l\x0b\xce\x00\x00\x00\x00\x00\x00'
Addesses   =  147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq
addr_match =  [('BTC.TOP', {'addr_match': '147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq'})]
cb_match   =  [('CANOE', {'cb_match': '/canoepool/'})]

Message    =  Addr and Cb match differ
Blockheight=  528055
Miner1     =  BTC.TOP
Miner2     =  CANOE
Coinbase   =  03b70e0841d6c9e7b743634c41d6c9e7b729aecb2f63616e6f65706f6f6c2fb700b90f1155010000000000
CoinbaseStr=  b'\x03\xb7\x0e\x08A\xd6\xc9\xe7\xb7CcLA\xd6\xc9\xe7\xb7)\xae\xcb/canoepool/\xb7\x00\xb9\x0f\x11U\x01\x00\x00\x00\x00\x00'
Addesses   =  147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq
addr_match =  [('BTC.TOP', {'addr_match': '147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aP

In [119]:
len(conflicts)

388

In [120]:
with open(miners_initial_btccom_conflicts_json_file, 'w') as outfile:
    json.dump(conflicts, outfile)

In [121]:
print("Added addresses by marker: ",util.check_for_miner_addresses_from_markers(miners_initial_btccom)) # 6067

Added addresses by marker:  6067


In [122]:
assert util.check_for_miner_addresses_from_markers(miners_initial_btccom) == 6067

In [123]:
assert util.check_for_obvious_address_collisions(miners_initial_btccom) == False

In [124]:
util.get_sample(blocks)

{'time': 1532819459,
 'cb': '038d2608232f706f6f6c2e626974636f696e2e636f6d2f5573652f426974636f696e2f436173682f1095c164009f220de4972cc6de64dd2200',
 'addresses': ['1NvWFTai59nRUGJyL8d5KVp83pVCY5Xv6A'],
 'miner': '',
 'conflicts': 0,
 'attribution': '',
 'attributions': {'blockchain_info_marker': {'miner': 'Bitcoin.com',
   'matches': [{'cb_match': 'pool.bitcoin.com'}],
   'src': 'blockchain.info'},
  'blockchain_info': {'miner': 'Bitcoin.com',
   'matches': [{'cb_match': 'pool.bitcoin.com'}],
   'src': 'blockchain.info'},
  'blockchain_info_address_update': {'miner': 'Bitcoin.com',
   'matches': [{'addr_match': '1NvWFTai59nRUGJyL8d5KVp83pVCY5Xv6A'}],
   'src': 'blockchain.info'},
  'blockchain_info_marker_update': {'miner': 'Bitcoin.com',
   'matches': [{'cb_match': 'pool.bitcoin.com'}],
   'src': 'blockchain.info'},
  'blockchain_info_update': {'miner': 'Bitcoin.com',
   'matches': [{'cb_match': 'pool.bitcoin.com'},
    {'addr_match': '1NvWFTai59nRUGJyL8d5KVp83pVCY5Xv6A'}],
   'src': 'b

## Miner unification between blockchain.info and btccom and enrich miners file from btccom

In [125]:
# Boostrap pools files with blockchain.info github data
# https://github.com/blockchain/Blockchain-Known-Pools/tree/82ed31956388e3950845cc2faeaf6679a057ee5b
miners = dict()
miners.clear()

miners = add_makers_from_source(pools_bc,miners,source=["blockchain.info github",])

Multiple markers for pool: OzCoin
Multiple markers for pool: TripleMining
Multiple markers for pool: Polmine
Multiple markers for pool: AntPool
Multiple markers for pool: BTCC Pool
Multiple markers for pool: BTCC Pool
Multiple markers for pool: BTCC Pool
Multiple markers for pool: BitFury
Multiple markers for pool: ViaBTC
Multiple markers for pool: PHash.IO


In [126]:
diffs = list(dictdiffer.diff(pools_bc,pools_btccom))
for diff in diffs:
    if diff[0] == "change":
        if "name" in diff[1]:
            #pprint.pprint(diff)
            util.add_name(miner=diff[2][0],miners=miners,name=diff[2][1],source="btccom github",currencies=["BTC",])

In [127]:
assert util.get_miner_id_by_name(miners,'NiceHash') == 'NiceHash Solo'

In [128]:
miners = add_makers_from_source(pools_btccom,miners,source=["btccom github",])

duplicate name in markers: CanoePool - CANOE
Multiple markers for pool: CANOE
Same marker for pool: BTC.TOP marker:/BTC.TOP/
Same marker for pool: Bitcoin.com marker:pool.bitcoin.com
Same marker for pool: 175btc marker:Mined By 175btc.com
Same marker for pool: GBMiners marker:/mined by gbminers/
Same marker for pool: A-XBT marker:/A-XBT/
Same marker for pool: ASICMiner marker:ASICMiner
Same marker for pool: BitMinter marker:BitMinter
Same marker for pool: BitcoinRussia marker:/Bitcoin-Russia.ru/
Same marker for pool: BTCServ marker:btcserv
Same marker for pool: simplecoin.us marker:simplecoin
Same marker for pool: BTC Guild marker:BTC Guild
Same marker for pool: Eligius marker:Eligius
Same marker for pool: OzCoin marker:ozco.in
Same marker for pool: OzCoin marker:ozcoin
Same marker for pool: EclipseMC marker:EMC
Same marker for pool: MaxBTC marker:MaxBTC
Same marker for pool: TripleMining marker:triplemining
Same marker for pool: TripleMining marker:Triplemining.com
Same marker for poo

In [129]:
assert "BWPool" in miners["BW.COM"][ util.D_NAMES ].keys()

In [130]:
miners = add_addresses_from_source(pools_bc,miners,source=["blockchain.info github",])

new pool: BTC Nuggets
new pool: EkanemBTC
new pool: Huobi
new pool: CloudHashing
new pool: digitalX Mintsy
new pool: Telco 214
new pool: BTC Pool Party
new pool: Multipool
new pool: transactioncoinmining
new pool: BTCDig
new pool: Tricky's BTC Pool
new pool: BTCMP
new pool: Eobot
new pool: UNOMP
new pool: Patel's Mining pool
new pool: GoGreenLight


In [131]:
miners["Huobi.pool"]

{'names': {'Huobi.pool': {'url': 'https://www.poolhb.com/',
   'currencies': ['BTC'],
   'fullName': '',
   'firstUsed': 0,
   'lastUsed': 0,
   'sources': ['btccom github', 'blockchain.info github']}},
 'markers': {'/Huobi/': {'currencies': ['BTC'],
   'firstUsed': 0,
   'lastUsed': 0,
   'sources': ['btccom github', 'blockchain.info github']}},
 'addresses': {}}

In [132]:
miners["Huobi"]

{'names': {'Huobi': {'url': 'http://www.huobi.com',
   'currencies': ['BTC'],
   'fullName': '',
   'firstUsed': 0,
   'lastUsed': 0,
   'sources': ['blockchain.info github']}},
 'markers': {},
 'addresses': {'3HuobiNg2wHjdPU2mQczL9on8WF7hZmaGd': {'currencies': ['BTC'],
   'firstUsed': 0,
   'lastUsed': 0,
   'sources': ['blockchain.info github']}}}

In [133]:
util.unify_miners("Huobi.pool","Huobi",miners)
print()

In [134]:
miners = add_addresses_from_source(pools_btccom,miners,source=["btccom github",])

CONFLICTING URL: 'https://www.bitfarms.io/'
CONFLICTING URL: 'https://www.bitarms.io/'
duplicate name in addresses: BWPool - BW.COM
CONFLICTING URL: 'https://bwpool.net/'
CONFLICTING URL: 'https://bw.com'
duplicate name in addresses: BitClub - BitClub Network
duplicate name in addresses: BTCC - BTCC Pool
new pool: Patels
duplicate name in addresses: BitcoinIndia - Bitcoin India
CONFLICTING URL: 'https://pool.bitcoin-india.org/'
CONFLICTING URL: 'https://bitcoin-india.org'
duplicate name in addresses: CanoePool - CANOE
CONFLICTING URL: 'https://btc.canoepool.com/'
CONFLICTING URL: 'https://www.canoepool.com'
new pool: tiger


In [135]:
miners["DCExploration"]

{'names': {'DCExploration': {'url': '',
   'currencies': ['BTC'],
   'fullName': '',
   'firstUsed': 0,
   'lastUsed': 0,
   'sources': ['blockchain.info github', 'btccom github']}},
 'markers': {'/DCExploration/': {'currencies': ['BTC'],
   'firstUsed': 0,
   'lastUsed': 0,
   'sources': ['blockchain.info github', 'btccom github']}},
 'addresses': {}}

In [136]:
miners["DCEX"]

{'names': {'DCEX': {'url': 'http://dcexploration.cn',
   'currencies': ['BTC'],
   'fullName': '',
   'firstUsed': 0,
   'lastUsed': 0,
   'sources': ['btccom github', 'blockchain.info github']}},
 'markers': {'/DCEX/': {'currencies': ['BTC'],
   'firstUsed': 0,
   'lastUsed': 0,
   'sources': ['btccom github', 'blockchain.info github']}},
 'addresses': {}}

In [137]:
util.unify_miners("DCEX","DCExploration",miners)
print()

In [138]:
miners["sigmapool.com"]

{'names': {'sigmapool.com': {'url': 'https://sigmapool.com',
   'currencies': ['BTC'],
   'fullName': '',
   'firstUsed': 0,
   'lastUsed': 0,
   'sources': ['btccom github', 'blockchain.info github']}},
 'markers': {'/SigmaPool.com/': {'currencies': ['BTC'],
   'firstUsed': 0,
   'lastUsed': 0,
   'sources': ['btccom github', 'blockchain.info github']}},
 'addresses': {}}

In [139]:
miners["SigmaPool.com"]

{'names': {'SigmaPool.com': {'url': 'https://www.sigmapool.com/',
   'currencies': ['BTC'],
   'fullName': '',
   'firstUsed': 0,
   'lastUsed': 0,
   'sources': ['blockchain.info github', 'btccom github']}},
 'markers': {'SigmaPool.com': {'currencies': ['BTC'],
   'firstUsed': 0,
   'lastUsed': 0,
   'sources': ['blockchain.info github', 'btccom github']}},
 'addresses': {}}

In [140]:
util.unify_miners("sigmapool.com","SigmaPool.com",miners)
print()

In [141]:
miners["tiger"]

{'names': {'tiger': {'url': '',
   'currencies': ['BTC'],
   'fullName': '',
   'firstUsed': 0,
   'lastUsed': 0,
   'sources': ['btccom github']}},
 'markers': {},
 'addresses': {'1LsFmhnne74EmU4q4aobfxfrWY4wfMVd8w': {'currencies': ['BTC'],
   'firstUsed': 0,
   'lastUsed': 0,
   'sources': ['btccom github']}}}

In [142]:
miners["Patels"]

{'names': {'Patels': {'url': 'http://patelsminingpool.com/',
   'currencies': ['BTC'],
   'fullName': '',
   'firstUsed': 0,
   'lastUsed': 0,
   'sources': ['btccom github']}},
 'markers': {},
 'addresses': {'19RE4mz2UbDxDVougc6GGdoT4x5yXxwFq2': {'currencies': ['BTC'],
   'firstUsed': 0,
   'lastUsed': 0,
   'sources': ['btccom github']},
  '197miJmttpCt2ubVs6DDtGBYFDroxHmvVB': {'currencies': ['BTC'],
   'firstUsed': 0,
   'lastUsed': 0,
   'sources': ['btccom github']}}}

In [143]:
assert "BitcoinIndia" in miners["Bitcoin India"][ util.D_NAMES ].keys()
assert "CanoePool" in miners["CANOE"][ util.D_NAMES ].keys()

In [144]:
util.get_sample(miners)

{'names': {'Polmine': {'url': 'https://polmine.pl',
   'currencies': ['BTC'],
   'fullName': '',
   'firstUsed': 0,
   'lastUsed': 0,
   'sources': ['blockchain.info github', 'btccom github']}},
 'markers': {'by polmine.pl': {'currencies': ['BTC'],
   'firstUsed': 0,
   'lastUsed': 0,
   'sources': ['blockchain.info github', 'btccom github']},
  'bypmneU': {'currencies': ['BTC'],
   'firstUsed': 0,
   'lastUsed': 0,
   'sources': ['blockchain.info github', 'btccom github']}},
 'addresses': {'17kkmDx8eSwj2JTTULb3HkJhCmexfysExz': {'currencies': ['BTC'],
   'firstUsed': 0,
   'lastUsed': 0,
   'sources': ['blockchain.info github', 'btccom github']},
  '1AajKXkaq2DsnDmP8ZPTrE5gH1HFo1x3AU': {'currencies': ['BTC'],
   'firstUsed': 0,
   'lastUsed': 0,
   'sources': ['blockchain.info github', 'btccom github']},
  '16cv7wyeG6RRqhvJpY21CnsjxuKj2gAoK2': {'currencies': ['BTC'],
   'firstUsed': 0,
   'lastUsed': 0,
   'sources': ['blockchain.info github', 'btccom github']},
  '13vWXwzNF5Ef9SUXNTdr

## Miner unification between blockchain.info and blocktrail.com

In [145]:
with open(blocks_blocktrail_json_file, 'r') as fp:
    blocks_blocktrail = json.load(fp)

In [146]:
blocks_blocktrail["400000"]

{'miner': 'BW Pool', 'src': 'blocktrail.com'}

Try to match every pool/miner name in the *blocktrail.com* attribution to a name from the *blockchain.info* attribution

In [147]:
miners_bci = set()
miners_bt = set()
miners_both = set()

for blknum in blocks:
    if blknum in blocks_blocktrail.keys():
        # check if there is a blocktrail.com attribution for this block
        if ( util.DD_BCI_ATTR in blocks[ blknum ][ util.D_ATTRIBUTIONS ].keys() or
             util.DD_BCI_MARK_ATTR in blocks[ blknum ][ util.D_ATTRIBUTIONS ].keys() or
             util.DD_BCI_ADDR_ATTR in blocks[ blknum ][ util.D_ATTRIBUTIONS ].keys() ):
            # check if there is a blockchain.info attribution already
            
            # assign blockchain.info attribuiton:
            if util.DD_BCI_ATTR in blocks[ blknum ][ util.D_ATTRIBUTIONS ].keys():
                miner_bci = blocks[ blknum ][ util.D_ATTRIBUTIONS ][ util.DD_BCI_ATTR ][ util.DDD_MINER ]
                src_bci = blocks[ blknum ][ util.D_ATTRIBUTIONS ][ util.DD_BCI_ATTR ][ util.DDD_SRC ]
            elif util.DD_BCI_MARK_ATTR in blocks[ blknum ][ util.D_ATTRIBUTIONS ].keys():
                miner_bci = blocks[ blknum ][ util.D_ATTRIBUTIONS ][ util.DD_BCI_MARK_ATTR ][ util.DDD_MINER ]
                src_bci = blocks[ blknum ][ util.D_ATTRIBUTIONS ][ util.DD_BCI_MARK_ATTR ][ util.DDD_SRC ]
            elif util.DD_BCI_ADDR_ATTR in blocks[ blknum ][ util.D_ATTRIBUTIONS ].keys():
                miner_bci = blocks[ blknum ][ util.D_ATTRIBUTIONS ][ util.DD_BCI_ADDR_ATTR ][ util.DDD_MINER ]
                src_bci = blocks[ blknum ][ util.D_ATTRIBUTIONS ][ util.DD_BCI_ADDR_ATTR ][ util.DDD_SRC ]
            
            # assign blocktrail.com attribution:
            miner_bt = blocks_blocktrail[ blknum ][ util.DDD_MINER ]
            src_bt = blocks_blocktrail[ blknum ][ util.DDD_SRC ]
            
            if miner_bt == "unknown" or miner_bt == "Unknown Entity":
                # only try to match known miners 
                continue
            
            # try some simple name matching of pool/miner:
            if miner_bci == miner_bt:
                miners_both.add(miner_bci)
                
            elif miner_bci.lower() == miner_bt.lower():
                miners_both.add(miner_bci.lower())
                
                util.add_name( miner_bci, miners, miner_bci.lower(), "blocktrail.com" )
                util.add_name( miner_bci, miners, miner_bt, "blocktrail.com" ) 
                
            elif re.sub('[^a-z0-9]','',miner_bci.lower() ) == re.sub('[^a-z0-9]','',miner_bt.lower() ):
                miners_both.add( re.sub('[^a-z0-9]', '', miner_bci.lower() ) )
                
                util.add_name( miner_bci, miners, re.sub('[^a-z0-9]','',miner_bci.lower() ), "blocktrail.com" )
                util.add_name( miner_bci, miners, miner_bt, "blocktrail.com" )
                
            elif ( re.sub( 'pool', '', re.sub('[^a-z0-9]','',miner_bci.lower() ) ) == 
                   re.sub( 'pool', '', re.sub('[^a-z0-9]','',miner_bt.lower() ) ) ):
                miners_both.add( re.sub( 'pool', '', re.sub('[^a-z0-9]', '', miner_bci.lower() ) ) )
                
                util.add_name( miner_bci, miners, re.sub( 'pool', '', re.sub('[^a-z0-9]','',miner_bci.lower() ) ), "blocktrail.com" )
                util.add_name( miner_bci, miners, miner_bt, "blocktrail.com" )
            else:
                # if no name has matched yet and no other miner matches directly add both names to a list 
                # to relsove later manually
                if miner_bt not in miners.keys():
                    miners_bci.add(miner_bci)
                    miners_bt.add(miner_bt)


In [148]:
len(sorted(miners_both))

59

In [149]:
assert len(sorted(miners_both)) == 59

In [150]:
sorted(miners_bci)

['BW.COM',
 'Bitcoin Affiliate Network',
 'Bixin',
 'F2Pool',
 'Huobi',
 'KanoPool',
 'Yourbtc.net',
 'simplecoin.us',
 'xbtc.exx.com&bw.com']

In [151]:
assert len(miners_bci) == 9

In [152]:
sorted(miners_bt)

['Avalon + Huobi',
 'BTCC',
 'BW Pool',
 'BitAffNet',
 'DiscusFish / F2Pool',
 'EXX & BW',
 'HaoBTC',
 'Kano CKPool',
 'Simplecoin',
 'YourBTC']

In [153]:
assert len(miners_bt) == 10

Manually fix pool/miner names that are not recognized automatically by adding them to the *names* dict of a *miner*:

In [154]:
# manual miner name fixes

# merge pool names:
util.add_name( "Huobi", miners, 'Avalon + Huobi', "blocktrail.com" )

util.add_name( "BTCC Pool", miners, 'BTCC', "blocktrail.com" )

util.add_name( "BW.COM", miners, 'BW Pool', "blocktrail.com" )

util.add_name( "Bitcoin Affiliate Network", miners, 'BitAffNet', "blocktrail.com" )

util.add_name( "CANOE", miners, 'Canoe Pool', "blocktrail.com" )

util.add_name( "F2Pool", miners, 'DiscusFish / F2Pool', "blocktrail.com" )
util.add_name( "F2Pool", miners, 'DiscusFish', "blocktrail.com" )

util.add_name( "xbtc.exx.com&bw.com", miners, "EXX & BW", "blocktrail.com" )

util.add_name( "Bixin", miners, "HaoBTC", "blocktrail.com" )
names_dict= { "HaoBTC": {  util.DD_URL:"https://haobtc.com/",
                           util.DD_CURRENCIES: ["BTC",],
                           util.DD_FULLNAME: "",
                           util.DD_FIRSTUSED: 0,
                           util.DD_LASTUSED: 0,
                           util.DD_SOURCES:["blocktrail.com",] } }
util.add_miner( "Bixin", miners, names_dict, update=True )

util.add_name( "KanoPool", miners, "Kano CKPool", "blocktrail.com" )

util.add_name( "simplecoin.us", miners, "Simplecoin", "blocktrail.com" )

util.add_name( "Yourbtc.net", miners, "YourBTC", "blocktrail.com" )

# add non-machted pool name manually
util.add_name( "digitalX Mintsy", miners, "digitalX Mintsy", "blocktrail.com" )


{'names': {'digitalX Mintsy': {'url': 'https://www.mintsy.co',
   'currencies': ['BTC'],
   'fullName': '',
   'firstUsed': 0,
   'lastUsed': 0,
   'sources': ['blockchain.info github', 'btccom github', 'blocktrail.com']}},
 'markers': {},
 'addresses': {'1NY15MK947MLzmPUa2gL7UgyR8prLh2xfu': {'currencies': ['BTC'],
   'firstUsed': 0,
   'lastUsed': 0,
   'sources': ['blockchain.info github', 'btccom github', 'blocktrail.com']}}}

In [155]:
assert "KNCMiner" in miners["KnCMiner"][ util.D_NAMES ].keys()
assert "kncminer" in miners["KnCMiner"][ util.D_NAMES ].keys()

In [156]:
with open(miners_initial_json_file, 'w') as outfile:
    json.dump(miners, outfile)

In [157]:
miners_initial = copy.deepcopy(miners)

## blocktrail.com attribution with unified miner ids
Run blocktrail attribution again with unified names and attribute blocks according to blocktrail based on unified names

In [158]:
unknown_counter = 0
attribution_counter = 0 

for blknum in blocks:
    # iterate over all blocks once
    if blknum in blocks_blocktrail.keys():
        # check if block was attributed by blocktrail
        miner_bt = blocks_blocktrail[ blknum ][ util.DDD_MINER ]
        if miner_bt == "unknown" or miner_bt == "Unknown Entity":
            unknown_counter +=1 
            continue
            
        miner_uid = miner_bt
        mapped = False
        for miner_id in miners:
            # check if there is a unified miner id for this miner already in miners.json
            if miner_bt in miners[ miner_id ][ util.D_NAMES ].keys():
                # unified miner_id found
                miner_uid = miner_id
                mapped = True
                break
                
        if not mapped and len( blocks[ blknum ][ util.D_ADDRESSES ] ) == 1:
            # if miner name could not be mapped to list of existing miner names
            # it is probably a new miner and gets added as such
            address = blocks[ blknum ][ util.D_ADDRESSES ][0]
            print(miner_uid,":",address)
            attribution_counter += 1
            util.add_miner(miner_uid,
                     miners,
                     names_dict= { miner_uid: { util.DD_URL:"",
                                            util.DD_CURRENCIES: ["BTC",],
                                            util.DD_FULLNAME: "",
                                            util.DD_FIRSTUSED: 0,
                                            util.DD_LASTUSED: 0,
                                            util.DD_SOURCES:["blocktrail.com",] } },
                     addresses_dict= { address: { util.DD_CURRENCIES: ["BTC",],
                                              util.DD_FIRSTUSED: 0,
                                              util.DD_LASTUSED: 0,
                                              util.DD_SOURCES:["blocktrail.com",] } }, )
        elif mapped and len( blocks[ blknum ][ util.D_ADDRESSES ] ) == 1:
            # the the miner could be mapped then check if we already have the address 
            # of the coinbase output mapped to this miner 
            # (if there is only one address in the block coinbase output)
            # If we dont have the address add it, if we have it add "blocktrail.com" as source to the address
            address = blocks[ blknum ][ util.D_ADDRESSES ][0]
            """
            if address == "1FLH1SoLv4U68yUERhDiWzrJn5TggMqkaZ":
                # check for specific address 
                for attr in blocks[ blknum ][ util.D_ATTRIBUTIONS ].keys():
                    if blocks[ blknum ][ util.D_ATTRIBUTIONS ][ attr ][ "miner" ] != miner_uid:
                        print(blknum,":",blocks[ blknum ][ util.D_ATTRIBUTIONS ][ attr ][ "miner" ],"--",miner_uid)
            """            
            util.add_addr(miner_uid,miners,address,source="blocktrail.com",currencies=["BTC",])
            attribution_counter += 1
        
        # attribute block based on miner_uid in any case
        blocks[ blknum ][ util.D_ATTRIBUTIONS ][ util.DD_BT_ATTR ] = { util.DDD_MINER:miner_uid,
                                                                       util.DDD_SRC:"blocktrail.com" }       
        

DeepBit : 14RxyduJ3CJnCqMGo7Bbz43axv1wdFZ5yM
P2Pool.org : 15rLkQjruGDKnQpm8Y6RdvRJJ9BNKzbvEw
Satoshi Systems : 1E9UoAzRpnZeymLh4JnLjADEaVyG5xKWma
itzod : 1JaepCfDnErPTPA96HJr7kfHZXLsN4asmH
IceDrill : 12ej4RUwoszmQoKYyFg6Ej27L82xhFS5Ao
Poolz 4 you : 1HhZiLEY8YatYS1KywhgetZVvnZ6j3pA8z
TangPool : 12Taz8FFXQ3E2AGn3ZW1SZM5bLnYGX4xR6


In [159]:
len(blocks_blocktrail)

514240

In [160]:
attribution_counter

301026

In [161]:
unknown_counter

189135

## Check block attributions

In [162]:
blocks["0"]

{'time': 1231006505,
 'cb': '04ffff001d0104455468652054696d65732030332f4a616e2f32303039204368616e63656c6c6f72206f6e206272696e6b206f66207365636f6e64206261696c6f757420666f722062616e6b73',
 'addresses': ['1A1zP1eP5QGefi2DMPTfTL5SLmv7DivfNa'],
 'miner': '',
 'conflicts': 0,
 'attribution': '',
 'attributions': {},
 'hash': '000000000019d6689c085ae165831e934ff763ae46a2a6c172b3f1b60a8ce26f',
 'phash': None,
 'payout': '5000000000'}

In [163]:
minblk = len(blocks) # check if start is really genesis at height 0 
maxblk = 0
for blknum in blocks.keys():
    if int(blknum) < minblk:
        minblk = int(blknum)
    if int(blknum) > maxblk:
        maxblk = int(blknum)
        
assert minblk == 0
assert maxblk == current_blockheight
print(minblk)
print(maxblk)

0
556400


### Analyze and resolve address/attribution conflicts 

Search for address conflicts where one address was attributed to more than one pool

In [164]:
# there should be 6 address conflicts now
util.check_for_obvious_address_collisions(miners)

1KFHE7w8BhaENAswwryaoccDb6qcT6DbYY : 2
1MimPd6LrPKGftPRHWdfk8S3KYBfN4ELnD : 2
147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq : 2
1FLH1SoLv4U68yUERhDiWzrJn5TggMqkaZ : 2
19RE4mz2UbDxDVougc6GGdoT4x5yXxwFq2 : 2
197miJmttpCt2ubVs6DDtGBYFDroxHmvVB : 2


True

In [165]:
address_conflicts_list = list()

remove_conflicts = False
address_conflicts = Counter()

for m in miners:
    for a in miners[ m ][ util.D_ADDRESSES ].keys():
        address_conflicts[ a ] += 1

for at in address_conflicts.most_common():
    if at[1] > 1:
        address_conflicts_list.append( at[0] )

#print(address_conflicts_list)

for a in address_conflicts_list:
    print("---")
    for m in miners:
        if a in miners[ m ][ util.D_ADDRESSES ].keys():
            print(a,":\nminer = ",m,"\n",miners[ m ][ util.D_ADDRESSES ][ a ],"\n")
            if remove_conflicts and 'blocktrail.com' in miners[ m ][ util.D_ADDRESSES ][ a ][ util.DD_SOURCES ]:
                miners[ m ][ util.D_ADDRESSES ].pop( a , None)


---
1KFHE7w8BhaENAswwryaoccDb6qcT6DbYY :
miner =  F2Pool 
 {'currencies': ['BTC'], 'firstUsed': 0, 'lastUsed': 0, 'sources': ['blockchain.info github', 'btccom github', 'blocktrail.com']} 

1KFHE7w8BhaENAswwryaoccDb6qcT6DbYY :
miner =  BTCC Pool 
 {'sources': ['blocktrail.com'], 'firstUsed': 0, 'lastUsed': 0, 'currencies': ['BTC']} 

---
1MimPd6LrPKGftPRHWdfk8S3KYBfN4ELnD :
miner =  digitalBTC 
 {'currencies': ['BTC'], 'firstUsed': 0, 'lastUsed': 0, 'sources': ['blockchain.info github', 'btccom github', 'blocktrail.com']} 

1MimPd6LrPKGftPRHWdfk8S3KYBfN4ELnD :
miner =  digitalX Mintsy 
 {'sources': ['blocktrail.com'], 'firstUsed': 0, 'lastUsed': 0, 'currencies': ['BTC']} 

---
147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq :
miner =  BTC.TOP 
 {'sources': ['blocktrail.com'], 'firstUsed': 0, 'lastUsed': 0, 'currencies': ['BTC']} 

147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq :
miner =  CANOE 
 {'sources': ['blocktrail.com'], 'firstUsed': 0, 'lastUsed': 0, 'currencies': ['BTC']} 

---
1FLH1SoLv4U68yUERhDiWzrJ

#### 1KFHE7w8BhaENAswwryaoccDb6qcT6DbYY

This address was a mis attribution by *Blocktrail.com* because the coinbase includes `btcc` and the Fish of discus fish.

In [166]:
for blknum in blocks:
    if blknum in blocks_blocktrail.keys() and len( blocks[ blknum ][ util.D_ADDRESSES ] ) == 1 and blocks[ blknum ][ util.D_ADDRESSES ][ 0 ] == "1KFHE7w8BhaENAswwryaoccDb6qcT6DbYY":
        miner_bt = blocks_blocktrail[ blknum ][ util.DDD_MINER ]
        miner_uid = ""
        for miner_id in miners:
            # check if there is a unified miner id for this miner already in miners.json
            if miner_bt in miners[ miner_id ][ util.D_NAMES ].keys():
                # unified miner_id found
                miner_uid = miner_id
                mapped = True
                break
        if mapped:        
            for attr in blocks[ blknum ][ util.D_ATTRIBUTIONS ].keys():
                if blocks[ blknum ][ util.D_ATTRIBUTIONS ][ attr ][ util.DDD_MINER ] != miner_uid:
                    print(blknum,": ",blocks[ blknum ][ util.D_ATTRIBUTIONS ][ attr ][ util.DDD_MINER ]," (",
                          blocks[ blknum ][ util.D_ATTRIBUTIONS ][ attr ][ util.DDD_SRC ],") -- ",
                          miner_uid," ( blocktrail.com )")
                    print("\t", repr(binascii.unhexlify(blocks[ blknum ][ util.D_CB ]) ) )
                    break

482886 :  F2Pool  ( blockchain.info ) --  BTCC Pool  ( blocktrail.com )
	 b'\x03F^\x07\x05/NYA/,\xfa\xbemm\xf8\x01\xe6\x8c4\xd5\x08\xa0\xf3\xd6u\xe1\x04\xe3l\xc4\xcfr0\xb1[V?\r\xbe\xd8\xff\xaf\xf0\xdc\xaf\xb1\x04\x00\x00\x00\xf0\x9f\x90\x9f\x10Mined by smcbtcc\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'


In [167]:
print(miners["BTCC Pool"][ util.D_ADDRESSES ][ "1KFHE7w8BhaENAswwryaoccDb6qcT6DbYY" ])
miners["BTCC Pool"][ util.D_ADDRESSES ].pop( "1KFHE7w8BhaENAswwryaoccDb6qcT6DbYY" , None)

{'sources': ['blocktrail.com'], 'firstUsed': 0, 'lastUsed': 0, 'currencies': ['BTC']}


{'sources': ['blocktrail.com'],
 'firstUsed': 0,
 'lastUsed': 0,
 'currencies': ['BTC']}

#### 1MimPd6LrPKGftPRHWdfk8S3KYBfN4ELnD
#### 19vvtxUpbidB8MT5CsSYYTBEjMRnowSZj4
#### 12f1FoTYvYiSmiDSVfeHcw8gS8Fp7xREUW
There are multiple occasions where *blocktrail.com* attributed blocks differently than *blockchain.info*
**digitalBTC** and **digitalX Mintsy**.
Accodring to an online search they really belong to the same company digitalBTC:
* https://www.coindesk.com/digitalbtc-launches-mining-contracts-platform-digitalx-mintsy/

In [168]:
i = 0
for blknum in blocks:
    if blknum in blocks_blocktrail.keys() and len( blocks[ blknum ][ util.D_ADDRESSES ] ) == 1 and blocks[ blknum ][ util.D_ADDRESSES ][ 0 ] == "1MimPd6LrPKGftPRHWdfk8S3KYBfN4ELnD":
        miner_bt = blocks_blocktrail[ blknum ][ util.DDD_MINER ]
        miner_uid = ""
        for miner_id in miners:
            # check if there is a unified miner id for this miner already in miners.json
            if miner_bt in miners[ miner_id ][ util.D_NAMES ].keys():
                # unified miner_id found
                miner_uid = miner_id
                mapped = True
                break
        if mapped:        
            for attr in blocks[ blknum ][ util.D_ATTRIBUTIONS ].keys():
                if blocks[ blknum ][ util.D_ATTRIBUTIONS ][ attr ][ util.DDD_MINER ] != miner_uid:
                    print(blknum,": ",blocks[ blknum ][ util.D_ATTRIBUTIONS ][ attr ][ util.DDD_MINER ]," (",
                          blocks[ blknum ][ util.D_ATTRIBUTIONS ][ attr ][ util.DDD_SRC ],") -- ",
                          miner_uid," ( blocktrail.com )")
                    print("\t", repr(binascii.unhexlify(blocks[ blknum ][ util.D_CB ]) ) )
                    i += 1
                    break
print(i)

298955 :  digitalBTC  ( blockchain.info ) --  digitalX Mintsy  ( blocktrail.com )
	 b"\x03\xcb\x8f\x04\x06/P2SH/\x04\xab!eS\x08@\x00\x00'\x0f\x91\x85\xca\r/nodeStratum/"
299011 :  digitalBTC  ( blockchain.info ) --  digitalX Mintsy  ( blocktrail.com )
	 b'\x03\x03\x90\x04\x06/P2SH/\x04\x95\xafeS\x08P\x00\x00\n\x10\xc5\x18@\r/nodeStratum/'
299228 :  digitalBTC  ( blockchain.info ) --  digitalX Mintsy  ( blocktrail.com )
	 b'\x03\xdc\x90\x04\x06/P2SH/\x04\xe9\xc1gS\x08p\x00\x00<\x0c\xb5\xb8d\r/nodeStratum/'
299240 :  digitalBTC  ( blockchain.info ) --  digitalX Mintsy  ( blocktrail.com )
	 b'\x03\xe8\x90\x04\x06/P2SH/\x04\x12\xd9gS\x08\x10\x00\x00a\x14\x1c\xec#\r/nodeStratum/'
299273 :  digitalBTC  ( blockchain.info ) --  digitalX Mintsy  ( blocktrail.com )
	 b'\x03\t\x91\x04\x06/P2SH/\x047-hS\x08 \x00\x00\x02\x15\xb7\xfd\x8b\r/nodeStratum/'
299274 :  digitalBTC  ( blockchain.info ) --  digitalX Mintsy  ( blocktrail.com )
	 b'\x03\n\x91\x04\x06/P2SH/\x04\x8a-hS\x08 \x00\x00 \x14`R\x7f\r/

74


In [169]:
print(miners["digitalX Mintsy"])
util.unify_miners("digitalX Mintsy","digitalBTC",miners)
print()

{'names': {'digitalX Mintsy': {'url': 'https://www.mintsy.co', 'currencies': ['BTC'], 'fullName': '', 'firstUsed': 0, 'lastUsed': 0, 'sources': ['blockchain.info github', 'btccom github', 'blocktrail.com']}}, 'markers': {}, 'addresses': {'1NY15MK947MLzmPUa2gL7UgyR8prLh2xfu': {'currencies': ['BTC'], 'firstUsed': 0, 'lastUsed': 0, 'sources': ['blockchain.info github', 'btccom github', 'blocktrail.com']}, '1MimPd6LrPKGftPRHWdfk8S3KYBfN4ELnD': {'sources': ['blocktrail.com'], 'firstUsed': 0, 'lastUsed': 0, 'currencies': ['BTC']}, '19vvtxUpbidB8MT5CsSYYTBEjMRnowSZj4': {'sources': ['blocktrail.com'], 'firstUsed': 0, 'lastUsed': 0, 'currencies': ['BTC']}, '12f1FoTYvYiSmiDSVfeHcw8gS8Fp7xREUW': {'sources': ['blocktrail.com'], 'firstUsed': 0, 'lastUsed': 0, 'currencies': ['BTC']}}}



#### 147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq
There are multiple occasions where *blocktrail.com* attributed blocks differently than *blockchain.info*:
**CANOE** and **BTC.TOP**.

With current information this cannot be resolved. 

In [170]:
i = 0
for blknum in blocks:
    if blknum in blocks_blocktrail.keys() and len( blocks[ blknum ][ util.D_ADDRESSES ] ) == 1 and blocks[ blknum ][ util.D_ADDRESSES ][ 0 ] == "147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq":
        miner_bt = blocks_blocktrail[ blknum ][ util.DDD_MINER ]
        miner_uid = ""
        for miner_id in miners:
            # check if there is a unified miner id for this miner already in miners.json
            if miner_bt in miners[ miner_id ][ util.D_NAMES ].keys():
                # unified miner_id found
                miner_uid = miner_id
                mapped = True
                break
        if mapped:        
            for attr in blocks[ blknum ][ util.D_ATTRIBUTIONS ].keys():
                if blocks[ blknum ][ util.D_ATTRIBUTIONS ][ attr ][ util.DDD_MINER ] != miner_uid:
                    print(blknum,": ",blocks[ blknum ][ util.D_ATTRIBUTIONS ][ attr ][ util.DDD_MINER ]," (",
                          blocks[ blknum ][ util.D_ATTRIBUTIONS ][ attr ][ util.DDD_SRC ],") -- ",
                          miner_uid," ( blocktrail.com )")
                    print("\t", repr(binascii.unhexlify(blocks[ blknum ][ util.D_CB ]) ) )
                    i += 1
                    break
print(i)

500540 :  BTC.TOP  ( blockchain.info ) --  CANOE  ( blocktrail.com )
	 b'\x03<\xa3\x07A\xd6\x8fC\xe9\xa4\x1a\x88A\xd6\x8fC\xe8\x8b\xe7\xf0/canoepool/\xba\x00\xb1\xc4o\x19\x01\x00\x00\x00\x00\x00'
500577 :  BTC.TOP  ( blockchain.info ) --  CANOE  ( blocktrail.com )
	 b"\x03a\xa3\x07A\xd6\x8fTK\x97klA\xd6\x8fTJp\xef\xbc/canoepool/\xba\x00\xc2\xa9#'\x01\x00\x00\x00\x00\x00"
500770 :  BTC.TOP  ( blockchain.info ) --  CANOE  ( blocktrail.com )
	 b'\x03"\xa4\x07A\xd6\x8f\xc5Q\xca\xe7MA\xd6\x8f\xc5Q\x1fs\xd9/canoepool/\xba\x00\xcaA\x94\xc8\x01\x00\x00\x00\x00\x00'
500855 :  BTC.TOP  ( blockchain.info ) --  CANOE  ( blocktrail.com )
	 b'\x03w\xa4\x07A\xd6\x8f\xf4\x10=\xeagA\xd6\x8f\xf4\x0f\xc2\x9e@/canoepool/\xba\x00\xaf\x01\x90|\x07\x00\x00\x00\x00\x00'
500886 :  BTC.TOP  ( blockchain.info ) --  CANOE  ( blocktrail.com )
	 b'\x03\x96\xa4\x07A\xd6\x90\x03\xfa\x80\x99\xfaA\xd6\x90\x03\xf9\xaah\x04/canoepool/\xb7\x00\xc19\n\xd8\x01\x00\x00\x00\x00\x00'
500971 :  BTC.TOP  ( blockchain.info ) --  

#### 1FLH1SoLv4U68yUERhDiWzrJn5TggMqkaZ
* 482059
* 482221

Waterhole -- BTC.com 

With current information this cannot be resovled

In [171]:
with open(miners_initial_incl_blocktrail_json_file, 'w') as outfile:
    json.dump(miners, outfile)

## Redo `blockchain.info` and `blocktrail.com` attributioins based on modified miner data
With updated miners 

In [172]:
miners = dict()
miners.clear()

with open(miners_initial_json_file, 'r') as fp:
    miners = json.load(fp)

In [173]:
# Update the miners initial but not persist them after that step
print(miners_initial["digitalX Mintsy"])
util.unify_miners("digitalX Mintsy","digitalBTC",miners_initial)

{'names': {'digitalX Mintsy': {'url': 'https://www.mintsy.co', 'currencies': ['BTC'], 'fullName': '', 'firstUsed': 0, 'lastUsed': 0, 'sources': ['blockchain.info github', 'btccom github', 'blocktrail.com']}}, 'markers': {}, 'addresses': {'1NY15MK947MLzmPUa2gL7UgyR8prLh2xfu': {'currencies': ['BTC'], 'firstUsed': 0, 'lastUsed': 0, 'sources': ['blockchain.info github', 'btccom github', 'blocktrail.com']}}}


{'SigmaPool.com': {'names': {'SigmaPool.com': {'url': 'https://www.sigmapool.com/',
    'currencies': ['BTC'],
    'fullName': '',
    'firstUsed': 0,
    'lastUsed': 0,
    'sources': ['blockchain.info github', 'btccom github', 'blocktrail.com']},
   'sigmapool.com': {'sources': ['btccom github', 'blockchain.info github'],
    'firstUsed': 0,
    'lastUsed': 0,
    'fullName': '',
    'currencies': ['BTC'],
    'url': 'https://sigmapool.com'}},
  'markers': {'SigmaPool.com': {'currencies': ['BTC'],
    'firstUsed': 0,
    'lastUsed': 0,
    'sources': ['blockchain.info github', 'btccom github', 'blocktrail.com']},
   '/SigmaPool.com/': {'sources': ['btccom github', 'blockchain.info github'],
    'firstUsed': 0,
    'lastUsed': 0,
    'currencies': ['BTC']}},
  'addresses': {}},
 'Bitcoin.com': {'names': {'Bitcoin.com': {'url': 'https://www.bitcoin.com',
    'currencies': ['BTC'],
    'fullName': '',
    'firstUsed': 0,
    'lastUsed': 0,
    'sources': ['blockchain.info github', 'btcc

In [174]:
util.get_sample(miners)

{'names': {'MultiCoin.co': {'url': 'http://multicoin.co',
   'currencies': ['BTC'],
   'fullName': '',
   'firstUsed': 0,
   'lastUsed': 0,
   'sources': ['blockchain.info github', 'btccom github', 'blocktrail.com']}},
 'markers': {'Mined by MultiCoin.co': {'currencies': ['BTC'],
   'firstUsed': 0,
   'lastUsed': 0,
   'sources': ['blockchain.info github', 'btccom github', 'blocktrail.com']}},
 'addresses': {}}

In [175]:
util.get_sample(blocks)

{'time': 1532636749,
 'cb': '033a2508043f2e5a5b612f4254432e434f4d2ffabe6d6db63e5e3074ef35e5070f945faf86e435f279cae7e452836c71daff17153a5304010000000000000071b56633afabaaca00000000',
 'addresses': ['1C1mCxRukix1KfegAY5zQQJV7samAciZpv'],
 'miner': '',
 'conflicts': 0,
 'attribution': '',
 'attributions': {'blockchain_info_marker': {'miner': 'BTC.com',
   'matches': [{'cb_match': '/BTC.COM/'}],
   'src': 'blockchain.info'},
  'blockchain_info': {'miner': 'BTC.com',
   'matches': [{'cb_match': '/BTC.COM/'}],
   'src': 'blockchain.info'},
  'blockchain_info_address_update': {'miner': 'BTC.com',
   'matches': [{'addr_match': '1C1mCxRukix1KfegAY5zQQJV7samAciZpv'}],
   'src': 'blockchain.info'},
  'blockchain_info_marker_update': {'miner': 'BTC.com',
   'matches': [{'cb_match': '/BTC.COM/'}],
   'src': 'blockchain.info'},
  'blockchain_info_update': {'miner': 'BTC.com',
   'matches': [{'cb_match': '/BTC.COM/'},
    {'addr_match': '1C1mCxRukix1KfegAY5zQQJV7samAciZpv'}],
   'src': 'blockchain.in

In [176]:
# attribute blocks to miners according to blockcahin.info initial mapping
(blocks,miners,conflicts) = util.attribute_blocks(blocks=blocks,
                                   miners_dict=miners,
                                   addr_attr="btccom_blockchain_info_addr",
                                   marker_attr="btccom_blockchain_info_marker",
                                   both_attr="btccom_blockchain_info",
                                   source="attribution based on btccom and blockchain.info sources",
                                   override=True,
                                   update=False)

482000
Message    =  Addr and Cb match differ
Blockheight=  482059
Miner1     =  Waterhole
Miner2     =  BTC.com
Coinbase   =  030b5b0704465fa1592f4254432e434f4d2ffabe6d6dff48161efa0f0b44fc9463e380e6b585ff91ac3f9c973d156dfff35fd553cdb101000000000000000228124b7faf010000000000
CoinbaseStr=  b'\x03\x0b[\x07\x04F_\xa1Y/BTC.COM/\xfa\xbemm\xffH\x16\x1e\xfa\x0f\x0bD\xfc\x94c\xe3\x80\xe6\xb5\x85\xff\x91\xac?\x9c\x97=\x15m\xff\xf3_\xd5S\xcd\xb1\x01\x00\x00\x00\x00\x00\x00\x00\x02(\x12K\x7f\xaf\x01\x00\x00\x00\x00\x00'
Addesses   =  1FLH1SoLv4U68yUERhDiWzrJn5TggMqkaZ
addr_match =  [('Waterhole', {'addr_match': '1FLH1SoLv4U68yUERhDiWzrJn5TggMqkaZ'})]
cb_match   =  [('BTC.com', {'cb_match': '/BTC.COM/'})]

Message    =  Addr and Cb match differ
Blockheight=  482221
Miner1     =  Waterhole
Miner2     =  BTC.com
Coinbase   =  03ad5b0704a4e6a2592f4254432e434f4d2ffabe6d6d0dd4f6c7dac894c84821c9fed8c15a621583ada8e4177c182c011cab52e2d6d0010000000000000001292f8e92ae010000000000
CoinbaseStr=  b'\x03\xad[\x

In [177]:
len(conflicts) 

3

In [178]:
# attribute blocks to miners according to blockcahin.info initial mapping
(blocks,miners,conflicts) = util.attribute_blocks(blocks=blocks,
                                   miners_dict=miners,
                                   addr_attr="btccom_blockchain_info_addr_update",
                                   marker_attr="btccom_blockchain_info_update",
                                   both_attr="btccom_blockchain_info_update",
                                   source="attribution based on btccom and blockchain.info sources",
                                   override=True,
                                   update=True)

160000
Message    =  Addr and Cb match differ
Blockheight=  160367
Miner1     =  OzCoin
Miner2     =  Yourbtc.net
Coinbase   =  70736a04ba760e1a046c730300522cfabe6d6dad6ee140dbdc015c8da85b8e3b636dc896e7df9203a7cd2c41e858503563ac130100000000000000796f75726274632e6e6574ac1eeeed88
CoinbaseStr=  b'psj\x04\xbav\x0e\x1a\x04ls\x03\x00R,\xfa\xbemm\xadn\xe1@\xdb\xdc\x01\\\x8d\xa8[\x8e;cm\xc8\x96\xe7\xdf\x92\x03\xa7\xcd,A\xe8XP5c\xac\x13\x01\x00\x00\x00\x00\x00\x00\x00yourbtc.net\xac\x1e\xee\xed\x88'
Addesses   =  1GG9HQZchCRxPSBV5SwZ9GoYEVq9vVLGqU
addr_match =  [('OzCoin', {'addr_match': '1GG9HQZchCRxPSBV5SwZ9GoYEVq9vVLGqU'})]
cb_match   =  [('Yourbtc.net', {'cb_match': 'yourbtc.net'})]

Message    =  Addr and Cb match differ
Blockheight=  160464
Miner1     =  OzCoin
Miner2     =  Yourbtc.net
Coinbase   =  70736a04ba760e1a04136c1100522cfabe6d6d7d24b129b8b97a267e531f9d816c6f435c9bb8a5fcc78b83360a5fee452c9e3e0100000000000000796f75726274632e6e6574ac1eeeed88
CoinbaseStr=  b'psj\x04\xbav\x0e\x1a\x04


Message    =  Addr and Cb match differ
Blockheight=  161514
Miner1     =  OzCoin
Miner2     =  Yourbtc.net
Coinbase   =  70736a04d7690d1a043c346400522cfabe6d6d9dbb4c7afc200cb164a63a37b638465b707637c239fa6151a192f7cc8b0691420100000000000000796f75726274632e6e6574ac1eeeed88
CoinbaseStr=  b'psj\x04\xd7i\r\x1a\x04<4d\x00R,\xfa\xbemm\x9d\xbbLz\xfc \x0c\xb1d\xa6:7\xb68F[pv7\xc29\xfaaQ\xa1\x92\xf7\xcc\x8b\x06\x91B\x01\x00\x00\x00\x00\x00\x00\x00yourbtc.net\xac\x1e\xee\xed\x88'
Addesses   =  1GG9HQZchCRxPSBV5SwZ9GoYEVq9vVLGqU
addr_match =  [('OzCoin', {'addr_match': '1GG9HQZchCRxPSBV5SwZ9GoYEVq9vVLGqU'})]
cb_match   =  [('Yourbtc.net', {'cb_match': 'yourbtc.net'})]

Message    =  Addr and Cb match differ
Blockheight=  161590
Miner1     =  OzCoin
Miner2     =  Yourbtc.net
Coinbase   =  70736a04d7690d1a044ec7ec00522cfabe6d6de99a3e314d36a6694de059bc711c3c004a70ecbdc04938769479d152c1b8595e0100000000000000796f75726274632e6e6574ac1eeeed88
CoinbaseStr=  b'psj\x04\xd7i\r\x1a\x04N\xc7\xec\x00R,\xfa\xbe

162000
Message    =  Addr and Cb match differ
Blockheight=  162024
Miner1     =  OzCoin
Miner2     =  Yourbtc.net
Coinbase   =  70736a04d7690d1a04f9aa3005522cfabe6d6d079ca0876cf5e21bfbdbae1b7a75b2aae290cebfe0493d57935c37250cb145460100000000000000796f75726274632e6e6574ac1eeeed88
CoinbaseStr=  b'psj\x04\xd7i\r\x1a\x04\xf9\xaa0\x05R,\xfa\xbemm\x07\x9c\xa0\x87l\xf5\xe2\x1b\xfb\xdb\xae\x1bzu\xb2\xaa\xe2\x90\xce\xbf\xe0I=W\x93\\7%\x0c\xb1EF\x01\x00\x00\x00\x00\x00\x00\x00yourbtc.net\xac\x1e\xee\xed\x88'
Addesses   =  1GG9HQZchCRxPSBV5SwZ9GoYEVq9vVLGqU
addr_match =  [('OzCoin', {'addr_match': '1GG9HQZchCRxPSBV5SwZ9GoYEVq9vVLGqU'})]
cb_match   =  [('Yourbtc.net', {'cb_match': 'yourbtc.net'})]

Message    =  Addr and Cb match differ
Blockheight=  162033
Miner1     =  OzCoin
Miner2     =  Yourbtc.net
Coinbase   =  70736a04d7690d1a04de9f4305522cfabe6d6dddd07348993994c5fef67c13a320f3303babe98124f79768eb3fe51626b66ae90100000000000000796f75726274632e6e6574ac1eeeed88
CoinbaseStr=  b'psj\x04\xd7i\r\x1

Message    =  Addr and Cb match differ
Blockheight=  162413
Miner1     =  OzCoin
Miner2     =  Yourbtc.net
Coinbase   =  70736a04d7690d1a04114aa307522cfabe6d6d37e4a5ea89722914cfd59779bf2121672af86f6749de1cc1f92dc6f024a21e050100000000000000796f75726274632e6e6574ac1eeeed88
CoinbaseStr=  b'psj\x04\xd7i\r\x1a\x04\x11J\xa3\x07R,\xfa\xbemm7\xe4\xa5\xea\x89r)\x14\xcf\xd5\x97y\xbf!!g*\xf8ogI\xde\x1c\xc1\xf9-\xc6\xf0$\xa2\x1e\x05\x01\x00\x00\x00\x00\x00\x00\x00yourbtc.net\xac\x1e\xee\xed\x88'
Addesses   =  1GG9HQZchCRxPSBV5SwZ9GoYEVq9vVLGqU
addr_match =  [('OzCoin', {'addr_match': '1GG9HQZchCRxPSBV5SwZ9GoYEVq9vVLGqU'})]
cb_match   =  [('Yourbtc.net', {'cb_match': 'yourbtc.net'})]

Message    =  Addr and Cb match differ
Blockheight=  162432
Miner1     =  OzCoin
Miner2     =  Yourbtc.net
Coinbase   =  70736a04d7690d1a04a939b807522cfabe6d6d05dcea2d4dc88d8a158fc767eea02d1ab019c262545606fc9939d7ba758065ea0100000000000000796f75726274632e6e6574ac1eeeed88
CoinbaseStr=  b'psj\x04\xd7i\r\x1a\x04\xa99\xb8

163000
Message    =  Addr and Cb match differ
Blockheight=  163013
Miner1     =  OzCoin
Miner2     =  Yourbtc.net
Coinbase   =  70736a04d7690d1a04fb784901522cfabe6d6d0b7b4b4c7d2a82e04142bc33676de5f1d6528066c0b79c1e58c1c3486e3aedc30100000000000000796f75726274632e6e6574ac1eeeed88
CoinbaseStr=  b'psj\x04\xd7i\r\x1a\x04\xfbxI\x01R,\xfa\xbemm\x0b{KL}*\x82\xe0AB\xbc3gm\xe5\xf1\xd6R\x80f\xc0\xb7\x9c\x1eX\xc1\xc3Hn:\xed\xc3\x01\x00\x00\x00\x00\x00\x00\x00yourbtc.net\xac\x1e\xee\xed\x88'
Addesses   =  1GG9HQZchCRxPSBV5SwZ9GoYEVq9vVLGqU
addr_match =  [('OzCoin', {'addr_match': '1GG9HQZchCRxPSBV5SwZ9GoYEVq9vVLGqU'})]
cb_match   =  [('Yourbtc.net', {'cb_match': 'yourbtc.net'})]

Message    =  Addr and Cb match differ
Blockheight=  163043
Miner1     =  OzCoin
Miner2     =  Yourbtc.net
Coinbase   =  70736a04d7690d1a04ea747501522cfabe6d6d3642c51f9b4f3a3ea0c72867929e3500d0346d75c8950e5f5c002c5c80909c700100000000000000796f75726274632e6e6574ac1eeeed88
CoinbaseStr=  b'psj\x04\xd7i\r\x1a\x04\xeatu\x01R,\x

...

Message    =  Addr and Cb match differ
Blockheight=  519347
Miner1     =  BTC.TOP
Miner2     =  CANOE
Coinbase   =  03b3ec0741d6b6fe62050e3941d6b6fe61dee2d42f63616e6f65706f6f6c2fb700e4a12395000000000000
CoinbaseStr=  b'\x03\xb3\xec\x07A\xd6\xb6\xfeb\x05\x0e9A\xd6\xb6\xfea\xde\xe2\xd4/canoepool/\xb7\x00\xe4\xa1#\x95\x00\x00\x00\x00\x00\x00'
Addesses   =  147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq
addr_match =  [('BTC.TOP', {'addr_match': '147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq'})]
cb_match   =  [('CANOE', {'cb_match': '/canoepool/'})]

Message    =  Addr and Cb match differ
Blockheight=  519452
Miner1     =  BTC.TOP
Miner2     =  CANOE
Coinbase   =  031ced0741d6b731729f64f941d6b731719b6f5b2f63616e6f65706f6f6c2fb70016ce3b66000000000000
CoinbaseStr=  b'\x03\x1c\xed\x07A\xd6\xb71r\x9fd\xf9A\xd6\xb71q\x9bo[/canoepool/\xb7\x00\x16\xce;f\x00\x00\x00\x00\x00\x00'
Addesses   =  147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq
addr_match =  [('BTC.TOP', {'addr_match': '147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq'})]
cb_match   = 

521000
Message    =  Addr and Cb match differ
Blockheight=  521032
Miner1     =  BTC.TOP
Miner2     =  CANOE
Coinbase   =  0348f30741d6bac12359b18641d6bac122cfe4062f63616e6f65706f6f6c2fb70052c57a70010000000000
CoinbaseStr=  b'\x03H\xf3\x07A\xd6\xba\xc1#Y\xb1\x86A\xd6\xba\xc1"\xcf\xe4\x06/canoepool/\xb7\x00R\xc5zp\x01\x00\x00\x00\x00\x00'
Addesses   =  147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq
addr_match =  [('BTC.TOP', {'addr_match': '147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq'})]
cb_match   =  [('CANOE', {'cb_match': '/canoepool/'})]

Message    =  Addr and Cb match differ
Blockheight=  521045
Miner1     =  BTC.TOP
Miner2     =  CANOE
Coinbase   =  0355f30741d6bac7fa4a70a741d6bac7fa1eea5c2f63616e6f65706f6f6c2fb70041f0fbb3010000000000
CoinbaseStr=  b'\x03U\xf3\x07A\xd6\xba\xc7\xfaJp\xa7A\xd6\xba\xc7\xfa\x1e\xea\\/canoepool/\xb7\x00A\xf0\xfb\xb3\x01\x00\x00\x00\x00\x00'
Addesses   =  147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq
addr_match =  [('BTC.TOP', {'addr_match': '147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq'})]
cb

523000
Message    =  Addr and Cb match differ
Blockheight=  523113
Miner1     =  BTC.TOP
Miner2     =  CANOE
Coinbase   =  0369fb0741d6bf5c1632db1441d6bf5c155bd9972f63616e6f65706f6f6c2fba0002024303004000000000
CoinbaseStr=  b'\x03i\xfb\x07A\xd6\xbf\\\x162\xdb\x14A\xd6\xbf\\\x15[\xd9\x97/canoepool/\xba\x00\x02\x02C\x03\x00@\x00\x00\x00\x00'
Addesses   =  147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq
addr_match =  [('BTC.TOP', {'addr_match': '147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq'})]
cb_match   =  [('CANOE', {'cb_match': '/canoepool/'})]

Message    =  Addr and Cb match differ
Blockheight=  523115
Miner1     =  BTC.TOP
Miner2     =  CANOE
Coinbase   =  036bfb0741d6bf5d32ffb0fa41d6bf5d325d22f92f63616e6f65706f6f6c2fb7004ce4bb6a000000000000
CoinbaseStr=  b'\x03k\xfb\x07A\xd6\xbf]2\xff\xb0\xfaA\xd6\xbf]2]"\xf9/canoepool/\xb7\x00L\xe4\xbbj\x00\x00\x00\x00\x00\x00'
Addesses   =  147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq
addr_match =  [('BTC.TOP', {'addr_match': '147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq'})]
cb_match   = 

Addesses   =  147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq
addr_match =  [('BTC.TOP', {'addr_match': '147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq'})]
cb_match   =  [('CANOE', {'cb_match': '/canoepool/'})]
526000
Message    =  Addr and Cb match differ
Blockheight=  526374
Miner1     =  BTC.TOP
Miner2     =  CANOE
Coinbase   =  0326080841d6c62e5d2da4de41d6c62e5c600f4c2f63616e6f65706f6f6c2fba008a9b4e6b000000000000
CoinbaseStr=  b'\x03&\x08\x08A\xd6\xc6.]-\xa4\xdeA\xd6\xc6.\\`\x0fL/canoepool/\xba\x00\x8a\x9bNk\x00\x00\x00\x00\x00\x00'
Addesses   =  147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq
addr_match =  [('BTC.TOP', {'addr_match': '147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq'})]
cb_match   =  [('CANOE', {'cb_match': '/canoepool/'})]

Message    =  Addr and Cb match differ
Blockheight=  526458
Miner1     =  BTC.TOP
Miner2     =  CANOE
Coinbase   =  037a080841d6c65c83e86f1541d6c65c83cbb2142f63616e6f65706f6f6c2fb7004501f0a6000000000000
CoinbaseStr=  b'\x03z\x08\x08A\xd6\xc6\\\x83\xe8o\x15A\xd6\xc6\\\x83\xcb\xb2\x14/canoepool/\

533000
Message    =  Addr and Cb match differ
Blockheight=  533832
Miner1     =  BTC.TOP
Miner2     =  CANOE
Coinbase   =  0348250841d6d6916c75c5bf41d6d6916b67abc22f63616e6f65706f6f6c2fba0037414edf000000000000
CoinbaseStr=  b'\x03H%\x08A\xd6\xd6\x91lu\xc5\xbfA\xd6\xd6\x91kg\xab\xc2/canoepool/\xba\x007AN\xdf\x00\x00\x00\x00\x00\x00'
Addesses   =  147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq
addr_match =  [('BTC.TOP', {'addr_match': '147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq'})]
cb_match   =  [('CANOE', {'cb_match': '/canoepool/'})]
534000
Message    =  Addr and Cb match differ
Blockheight=  534097
Miner1     =  BTC.TOP
Miner2     =  CANOE
Coinbase   =  0351260841d6d71de7305b7741d6d71de5fecc0d2f63616e6f65706f6f6c2fba003571e1cc010000000000
CoinbaseStr=  b'\x03Q&\x08A\xd6\xd7\x1d\xe70[wA\xd6\xd7\x1d\xe5\xfe\xcc\r/canoepool/\xba\x005q\xe1\xcc\x01\x00\x00\x00\x00\x00'
Addesses   =  147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq
addr_match =  [('BTC.TOP', {'addr_match': '147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq'})]
cb_match   

In [179]:
len(conflicts) # 479 # 455

388

In [180]:
miners = dict()
miners.clear()

with open(miners_initial_incl_blocktrail_json_file, 'r') as fp:
    miners = json.load(fp)

In [181]:
(blocks,miners,conflicts) = util.attribute_blocks(blocks=blocks,
                                   miners_dict=miners,
                                   addr_attr="initial_addr",
                                   marker_attr="initial_marker",
                                   both_attr="initial",
                                   source="initial attribution based on all sources",
                                   override=True,
                                   update=False)

159000
Message    =  Addr and Cb match differ
Blockheight=  159846
Miner1     =  Yourbtc.net
Miner2     =  OzCoin
Coinbase   =  70736a04ba760e1a040c9b0100522cfabe6d6dd5198c9a7f3796c9599592dc199792cc88c0a5588c2b9f5b491c1d8cbed5700601000000000000006f7a636f2e696eac1eeeed88
CoinbaseStr=  b'psj\x04\xbav\x0e\x1a\x04\x0c\x9b\x01\x00R,\xfa\xbemm\xd5\x19\x8c\x9a\x7f7\x96\xc9Y\x95\x92\xdc\x19\x97\x92\xcc\x88\xc0\xa5X\x8c+\x9f[I\x1c\x1d\x8c\xbe\xd5p\x06\x01\x00\x00\x00\x00\x00\x00\x00ozco.in\xac\x1e\xee\xed\x88'
Addesses   =  1GG9HQZchCRxPSBV5SwZ9GoYEVq9vVLGqU
addr_match =  [('Yourbtc.net', {'addr_match': '1GG9HQZchCRxPSBV5SwZ9GoYEVq9vVLGqU'})]
cb_match   =  [('OzCoin', {'cb_match': 'ozco.in'})]

Message    =  Addr and Cb match differ
Blockheight=  159929
Miner1     =  Yourbtc.net
Miner2     =  OzCoin
Coinbase   =  70736a04ba760e1a0418930700522cfabe6d6dc15059fd58be57a512008aff27237100949e3afd1e7f7be1dc5cc5346f7169b401000000000000006f7a636f2e696eac1eeeed88
CoinbaseStr=  b"psj\x04\xbav\x0e\x1a\x04\

473000
Message    =  Multiple addresses match
Blockheight=  473013
Miner1     =  BTC.com
Miner2     =  Waterhole
Coinbase   =  
CoinbaseStr=  b''
Addesses   =  1FLH1SoLv4U68yUERhDiWzrJn5TggMqkaZ
addr_match =  [('BTC.com', {'addr_match': '1FLH1SoLv4U68yUERhDiWzrJn5TggMqkaZ'}), ('Waterhole', {'addr_match': '1FLH1SoLv4U68yUERhDiWzrJn5TggMqkaZ'})]
cb_match   =  None

Message    =  Multiple addresses match
Blockheight=  473720
Miner1     =  BTC.com
Miner2     =  Waterhole
Coinbase   =  
CoinbaseStr=  b''
Addesses   =  1FLH1SoLv4U68yUERhDiWzrJn5TggMqkaZ
addr_match =  [('BTC.com', {'addr_match': '1FLH1SoLv4U68yUERhDiWzrJn5TggMqkaZ'}), ('Waterhole', {'addr_match': '1FLH1SoLv4U68yUERhDiWzrJn5TggMqkaZ'})]
cb_match   =  None

Message    =  Multiple addresses match
Blockheight=  473765
Miner1     =  BTC.com
Miner2     =  Waterhole
Coinbase   =  
CoinbaseStr=  b''
Addesses   =  1FLH1SoLv4U68yUERhDiWzrJn5TggMqkaZ
addr_match =  [('BTC.com', {'addr_match': '1FLH1SoLv4U68yUERhDiWzrJn5TggMqkaZ'}), ('Wat


Message    =  Multiple addresses match
Blockheight=  476509
Miner1     =  BTC.com
Miner2     =  Waterhole
Coinbase   =  
CoinbaseStr=  b''
Addesses   =  1FLH1SoLv4U68yUERhDiWzrJn5TggMqkaZ
addr_match =  [('BTC.com', {'addr_match': '1FLH1SoLv4U68yUERhDiWzrJn5TggMqkaZ'}), ('Waterhole', {'addr_match': '1FLH1SoLv4U68yUERhDiWzrJn5TggMqkaZ'})]
cb_match   =  None

Message    =  Multiple addresses match
Blockheight=  476522
Miner1     =  BTC.com
Miner2     =  Waterhole
Coinbase   =  
CoinbaseStr=  b''
Addesses   =  1FLH1SoLv4U68yUERhDiWzrJn5TggMqkaZ
addr_match =  [('BTC.com', {'addr_match': '1FLH1SoLv4U68yUERhDiWzrJn5TggMqkaZ'}), ('Waterhole', {'addr_match': '1FLH1SoLv4U68yUERhDiWzrJn5TggMqkaZ'})]
cb_match   =  None

Message    =  Multiple addresses match
Blockheight=  476708
Miner1     =  BTC.com
Miner2     =  Waterhole
Coinbase   =  
CoinbaseStr=  b''
Addesses   =  1FLH1SoLv4U68yUERhDiWzrJn5TggMqkaZ
addr_match =  [('BTC.com', {'addr_match': '1FLH1SoLv4U68yUERhDiWzrJn5TggMqkaZ'}), ('Waterhole

479000
Message    =  Multiple addresses match
Blockheight=  479111
Miner1     =  BTC.com
Miner2     =  Waterhole
Coinbase   =  
CoinbaseStr=  b''
Addesses   =  1FLH1SoLv4U68yUERhDiWzrJn5TggMqkaZ
addr_match =  [('BTC.com', {'addr_match': '1FLH1SoLv4U68yUERhDiWzrJn5TggMqkaZ'}), ('Waterhole', {'addr_match': '1FLH1SoLv4U68yUERhDiWzrJn5TggMqkaZ'})]
cb_match   =  None

Message    =  Multiple addresses match
Blockheight=  479262
Miner1     =  BTC.com
Miner2     =  Waterhole
Coinbase   =  
CoinbaseStr=  b''
Addesses   =  1FLH1SoLv4U68yUERhDiWzrJn5TggMqkaZ
addr_match =  [('BTC.com', {'addr_match': '1FLH1SoLv4U68yUERhDiWzrJn5TggMqkaZ'}), ('Waterhole', {'addr_match': '1FLH1SoLv4U68yUERhDiWzrJn5TggMqkaZ'})]
cb_match   =  None

Message    =  Multiple addresses match
Blockheight=  479339
Miner1     =  BTC.com
Miner2     =  Waterhole
Coinbase   =  
CoinbaseStr=  b''
Addesses   =  1FLH1SoLv4U68yUERhDiWzrJn5TggMqkaZ
addr_match =  [('BTC.com', {'addr_match': '1FLH1SoLv4U68yUERhDiWzrJn5TggMqkaZ'}), ('Wat

483000
Message    =  Multiple addresses match
Blockheight=  483576
Miner1     =  BTC.TOP
Miner2     =  CANOE
Coinbase   =  
CoinbaseStr=  b''
Addesses   =  147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq
addr_match =  [('BTC.TOP', {'addr_match': '147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq'}), ('CANOE', {'addr_match': '147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq'})]
cb_match   =  None
484000
Message    =  Multiple addresses match
Blockheight=  484105
Miner1     =  BTC.com
Miner2     =  Waterhole
Coinbase   =  
CoinbaseStr=  b''
Addesses   =  1FLH1SoLv4U68yUERhDiWzrJn5TggMqkaZ
addr_match =  [('BTC.com', {'addr_match': '1FLH1SoLv4U68yUERhDiWzrJn5TggMqkaZ'}), ('Waterhole', {'addr_match': '1FLH1SoLv4U68yUERhDiWzrJn5TggMqkaZ'})]
cb_match   =  None

Message    =  Multiple addresses match
Blockheight=  484488
Miner1     =  BTC.com
Miner2     =  Waterhole
Coinbase   =  
CoinbaseStr=  b''
Addesses   =  1FLH1SoLv4U68yUERhDiWzrJn5TggMqkaZ
addr_match =  [('BTC.com', {'addr_match': '1FLH1SoLv4U68yUERhDiWzrJn5TggMqkaZ'}), ('Water

...


Message    =  Multiple addresses match
Blockheight=  518623
Miner1     =  BTC.TOP
Miner2     =  CANOE
Coinbase   =  
CoinbaseStr=  b''
Addesses   =  147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq
addr_match =  [('BTC.TOP', {'addr_match': '147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq'}), ('CANOE', {'addr_match': '147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq'})]
cb_match   =  None

Message    =  Multiple addresses match
Blockheight=  518627
Miner1     =  BTC.TOP
Miner2     =  CANOE
Coinbase   =  
CoinbaseStr=  b''
Addesses   =  147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq
addr_match =  [('BTC.TOP', {'addr_match': '147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq'}), ('CANOE', {'addr_match': '147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq'})]
cb_match   =  None

Message    =  Multiple addresses match
Blockheight=  518652
Miner1     =  BTC.TOP
Miner2     =  CANOE
Coinbase   =  
CoinbaseStr=  b''
Addesses   =  147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq
addr_match =  [('BTC.TOP', {'addr_match': '147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq'}), ('CANOE', {'addr_match': '147Sw

520000
Message    =  Multiple addresses match
Blockheight=  520071
Miner1     =  BTC.TOP
Miner2     =  CANOE
Coinbase   =  
CoinbaseStr=  b''
Addesses   =  147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq
addr_match =  [('BTC.TOP', {'addr_match': '147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq'}), ('CANOE', {'addr_match': '147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq'})]
cb_match   =  None

Message    =  Multiple addresses match
Blockheight=  520149
Miner1     =  BTC.TOP
Miner2     =  CANOE
Coinbase   =  
CoinbaseStr=  b''
Addesses   =  147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq
addr_match =  [('BTC.TOP', {'addr_match': '147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq'}), ('CANOE', {'addr_match': '147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq'})]
cb_match   =  None

Message    =  Multiple addresses match
Blockheight=  520171
Miner1     =  BTC.TOP
Miner2     =  CANOE
Coinbase   =  
CoinbaseStr=  b''
Addesses   =  147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq
addr_match =  [('BTC.TOP', {'addr_match': '147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq'}), ('CANOE', {'addr_match': 

522000
Message    =  Multiple addresses match
Blockheight=  522015
Miner1     =  BTC.TOP
Miner2     =  CANOE
Coinbase   =  
CoinbaseStr=  b''
Addesses   =  147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq
addr_match =  [('BTC.TOP', {'addr_match': '147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq'}), ('CANOE', {'addr_match': '147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq'})]
cb_match   =  None

Message    =  Multiple addresses match
Blockheight=  522083
Miner1     =  BTC.TOP
Miner2     =  CANOE
Coinbase   =  
CoinbaseStr=  b''
Addesses   =  147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq
addr_match =  [('BTC.TOP', {'addr_match': '147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq'}), ('CANOE', {'addr_match': '147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq'})]
cb_match   =  None

Message    =  Multiple addresses match
Blockheight=  522147
Miner1     =  BTC.TOP
Miner2     =  CANOE
Coinbase   =  
CoinbaseStr=  b''
Addesses   =  147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq
addr_match =  [('BTC.TOP', {'addr_match': '147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq'}), ('CANOE', {'addr_match': 


Message    =  Multiple addresses match
Blockheight=  523838
Miner1     =  BTC.TOP
Miner2     =  CANOE
Coinbase   =  
CoinbaseStr=  b''
Addesses   =  147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq
addr_match =  [('BTC.TOP', {'addr_match': '147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq'}), ('CANOE', {'addr_match': '147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq'})]
cb_match   =  None
524000
Message    =  Multiple addresses match
Blockheight=  524017
Miner1     =  BTC.TOP
Miner2     =  CANOE
Coinbase   =  
CoinbaseStr=  b''
Addesses   =  147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq
addr_match =  [('BTC.TOP', {'addr_match': '147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq'}), ('CANOE', {'addr_match': '147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq'})]
cb_match   =  None

Message    =  Multiple addresses match
Blockheight=  524023
Miner1     =  BTC.TOP
Miner2     =  CANOE
Coinbase   =  
CoinbaseStr=  b''
Addesses   =  147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq
addr_match =  [('BTC.TOP', {'addr_match': '147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq'}), ('CANOE', {'addr_match': 


Message    =  Multiple addresses match
Blockheight=  527627
Miner1     =  BTC.TOP
Miner2     =  CANOE
Coinbase   =  
CoinbaseStr=  b''
Addesses   =  147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq
addr_match =  [('BTC.TOP', {'addr_match': '147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq'}), ('CANOE', {'addr_match': '147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq'})]
cb_match   =  None

Message    =  Multiple addresses match
Blockheight=  527658
Miner1     =  BTC.TOP
Miner2     =  CANOE
Coinbase   =  
CoinbaseStr=  b''
Addesses   =  147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq
addr_match =  [('BTC.TOP', {'addr_match': '147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq'}), ('CANOE', {'addr_match': '147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq'})]
cb_match   =  None
528000
Message    =  Multiple addresses match
Blockheight=  528047
Miner1     =  BTC.TOP
Miner2     =  CANOE
Coinbase   =  
CoinbaseStr=  b''
Addesses   =  147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq
addr_match =  [('BTC.TOP', {'addr_match': '147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq'}), ('CANOE', {'addr_match': 

In [182]:
len(conflicts)

455

In [183]:
(blocks,miners,conflicts) = util.attribute_blocks(blocks=blocks,
                                   miners_dict=miners,
                                   addr_attr="initial_addr_update",
                                   marker_attr="initial_marker_update",
                                   both_attr="initial_update",
                                   source="initial attribution based on all sources including update",
                                   override=True,
                                   update=True)

159000
Message    =  Addr and Cb match differ
Blockheight=  159846
Miner1     =  Yourbtc.net
Miner2     =  OzCoin
Coinbase   =  70736a04ba760e1a040c9b0100522cfabe6d6dd5198c9a7f3796c9599592dc199792cc88c0a5588c2b9f5b491c1d8cbed5700601000000000000006f7a636f2e696eac1eeeed88
CoinbaseStr=  b'psj\x04\xbav\x0e\x1a\x04\x0c\x9b\x01\x00R,\xfa\xbemm\xd5\x19\x8c\x9a\x7f7\x96\xc9Y\x95\x92\xdc\x19\x97\x92\xcc\x88\xc0\xa5X\x8c+\x9f[I\x1c\x1d\x8c\xbe\xd5p\x06\x01\x00\x00\x00\x00\x00\x00\x00ozco.in\xac\x1e\xee\xed\x88'
Addesses   =  1GG9HQZchCRxPSBV5SwZ9GoYEVq9vVLGqU
addr_match =  [('Yourbtc.net', {'addr_match': '1GG9HQZchCRxPSBV5SwZ9GoYEVq9vVLGqU'})]
cb_match   =  [('OzCoin', {'cb_match': 'ozco.in'})]

Message    =  Addr and Cb match differ
Blockheight=  159929
Miner1     =  Yourbtc.net
Miner2     =  OzCoin
Coinbase   =  70736a04ba760e1a0418930700522cfabe6d6dc15059fd58be57a512008aff27237100949e3afd1e7f7be1dc5cc5346f7169b401000000000000006f7a636f2e696eac1eeeed88
CoinbaseStr=  b"psj\x04\xbav\x0e\x1a\x04\

473000
Message    =  Multiple addresses match
Blockheight=  473013
Miner1     =  BTC.com
Miner2     =  Waterhole
Coinbase   =  
CoinbaseStr=  b''
Addesses   =  1FLH1SoLv4U68yUERhDiWzrJn5TggMqkaZ
addr_match =  [('BTC.com', {'addr_match': '1FLH1SoLv4U68yUERhDiWzrJn5TggMqkaZ'}), ('Waterhole', {'addr_match': '1FLH1SoLv4U68yUERhDiWzrJn5TggMqkaZ'})]
cb_match   =  None

Message    =  Multiple addresses match
Blockheight=  473720
Miner1     =  BTC.com
Miner2     =  Waterhole
Coinbase   =  
CoinbaseStr=  b''
Addesses   =  1FLH1SoLv4U68yUERhDiWzrJn5TggMqkaZ
addr_match =  [('BTC.com', {'addr_match': '1FLH1SoLv4U68yUERhDiWzrJn5TggMqkaZ'}), ('Waterhole', {'addr_match': '1FLH1SoLv4U68yUERhDiWzrJn5TggMqkaZ'})]
cb_match   =  None

Message    =  Multiple addresses match
Blockheight=  473765
Miner1     =  BTC.com
Miner2     =  Waterhole
Coinbase   =  
CoinbaseStr=  b''
Addesses   =  1FLH1SoLv4U68yUERhDiWzrJn5TggMqkaZ
addr_match =  [('BTC.com', {'addr_match': '1FLH1SoLv4U68yUERhDiWzrJn5TggMqkaZ'}), ('Wat

Message    =  Multiple addresses match
Blockheight=  476522
Miner1     =  BTC.com
Miner2     =  Waterhole
Coinbase   =  
CoinbaseStr=  b''
Addesses   =  1FLH1SoLv4U68yUERhDiWzrJn5TggMqkaZ
addr_match =  [('BTC.com', {'addr_match': '1FLH1SoLv4U68yUERhDiWzrJn5TggMqkaZ'}), ('Waterhole', {'addr_match': '1FLH1SoLv4U68yUERhDiWzrJn5TggMqkaZ'})]
cb_match   =  None

Message    =  Multiple addresses match
Blockheight=  476708
Miner1     =  BTC.com
Miner2     =  Waterhole
Coinbase   =  
CoinbaseStr=  b''
Addesses   =  1FLH1SoLv4U68yUERhDiWzrJn5TggMqkaZ
addr_match =  [('BTC.com', {'addr_match': '1FLH1SoLv4U68yUERhDiWzrJn5TggMqkaZ'}), ('Waterhole', {'addr_match': '1FLH1SoLv4U68yUERhDiWzrJn5TggMqkaZ'})]
cb_match   =  None

Message    =  Multiple addresses match
Blockheight=  476850
Miner1     =  BTC.com
Miner2     =  Waterhole
Coinbase   =  
CoinbaseStr=  b''
Addesses   =  1FLH1SoLv4U68yUERhDiWzrJn5TggMqkaZ
addr_match =  [('BTC.com', {'addr_match': '1FLH1SoLv4U68yUERhDiWzrJn5TggMqkaZ'}), ('Waterhole'

479000
Message    =  Multiple addresses match
Blockheight=  479111
Miner1     =  BTC.com
Miner2     =  Waterhole
Coinbase   =  
CoinbaseStr=  b''
Addesses   =  1FLH1SoLv4U68yUERhDiWzrJn5TggMqkaZ
addr_match =  [('BTC.com', {'addr_match': '1FLH1SoLv4U68yUERhDiWzrJn5TggMqkaZ'}), ('Waterhole', {'addr_match': '1FLH1SoLv4U68yUERhDiWzrJn5TggMqkaZ'})]
cb_match   =  None

Message    =  Multiple addresses match
Blockheight=  479262
Miner1     =  BTC.com
Miner2     =  Waterhole
Coinbase   =  
CoinbaseStr=  b''
Addesses   =  1FLH1SoLv4U68yUERhDiWzrJn5TggMqkaZ
addr_match =  [('BTC.com', {'addr_match': '1FLH1SoLv4U68yUERhDiWzrJn5TggMqkaZ'}), ('Waterhole', {'addr_match': '1FLH1SoLv4U68yUERhDiWzrJn5TggMqkaZ'})]
cb_match   =  None

Message    =  Multiple addresses match
Blockheight=  479339
Miner1     =  BTC.com
Miner2     =  Waterhole
Coinbase   =  
CoinbaseStr=  b''
Addesses   =  1FLH1SoLv4U68yUERhDiWzrJn5TggMqkaZ
addr_match =  [('BTC.com', {'addr_match': '1FLH1SoLv4U68yUERhDiWzrJn5TggMqkaZ'}), ('Wat

483000
Message    =  Multiple addresses match
Blockheight=  483576
Miner1     =  BTC.TOP
Miner2     =  CANOE
Coinbase   =  
CoinbaseStr=  b''
Addesses   =  147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq
addr_match =  [('BTC.TOP', {'addr_match': '147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq'}), ('CANOE', {'addr_match': '147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq'})]
cb_match   =  None
484000
Message    =  Multiple addresses match
Blockheight=  484105
Miner1     =  BTC.com
Miner2     =  Waterhole
Coinbase   =  
CoinbaseStr=  b''
Addesses   =  1FLH1SoLv4U68yUERhDiWzrJn5TggMqkaZ
addr_match =  [('BTC.com', {'addr_match': '1FLH1SoLv4U68yUERhDiWzrJn5TggMqkaZ'}), ('Waterhole', {'addr_match': '1FLH1SoLv4U68yUERhDiWzrJn5TggMqkaZ'})]
cb_match   =  None

Message    =  Multiple addresses match
Blockheight=  484488
Miner1     =  BTC.com
Miner2     =  Waterhole
Coinbase   =  
CoinbaseStr=  b''
Addesses   =  1FLH1SoLv4U68yUERhDiWzrJn5TggMqkaZ
addr_match =  [('BTC.com', {'addr_match': '1FLH1SoLv4U68yUERhDiWzrJn5TggMqkaZ'}), ('Water

...


Message    =  Multiple addresses match
Blockheight=  519253
Miner1     =  BTC.TOP
Miner2     =  CANOE
Coinbase   =  
CoinbaseStr=  b''
Addesses   =  147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq
addr_match =  [('BTC.TOP', {'addr_match': '147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq'}), ('CANOE', {'addr_match': '147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq'})]
cb_match   =  None

Message    =  Multiple addresses match
Blockheight=  519265
Miner1     =  BTC.TOP
Miner2     =  CANOE
Coinbase   =  
CoinbaseStr=  b''
Addesses   =  147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq
addr_match =  [('BTC.TOP', {'addr_match': '147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq'}), ('CANOE', {'addr_match': '147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq'})]
cb_match   =  None

Message    =  Multiple addresses match
Blockheight=  519304
Miner1     =  BTC.TOP
Miner2     =  CANOE
Coinbase   =  
CoinbaseStr=  b''
Addesses   =  147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq
addr_match =  [('BTC.TOP', {'addr_match': '147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq'}), ('CANOE', {'addr_match': '147Sw

521000
Message    =  Multiple addresses match
Blockheight=  521032
Miner1     =  BTC.TOP
Miner2     =  CANOE
Coinbase   =  
CoinbaseStr=  b''
Addesses   =  147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq
addr_match =  [('BTC.TOP', {'addr_match': '147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq'}), ('CANOE', {'addr_match': '147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq'})]
cb_match   =  None

Message    =  Multiple addresses match
Blockheight=  521045
Miner1     =  BTC.TOP
Miner2     =  CANOE
Coinbase   =  
CoinbaseStr=  b''
Addesses   =  147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq
addr_match =  [('BTC.TOP', {'addr_match': '147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq'}), ('CANOE', {'addr_match': '147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq'})]
cb_match   =  None

Message    =  Multiple addresses match
Blockheight=  521242
Miner1     =  BTC.TOP
Miner2     =  CANOE
Coinbase   =  
CoinbaseStr=  b''
Addesses   =  147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq
addr_match =  [('BTC.TOP', {'addr_match': '147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq'}), ('CANOE', {'addr_match': 


Message    =  Multiple addresses match
Blockheight=  522653
Miner1     =  BTC.TOP
Miner2     =  CANOE
Coinbase   =  
CoinbaseStr=  b''
Addesses   =  147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq
addr_match =  [('BTC.TOP', {'addr_match': '147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq'}), ('CANOE', {'addr_match': '147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq'})]
cb_match   =  None

Message    =  Multiple addresses match
Blockheight=  522710
Miner1     =  BTC.TOP
Miner2     =  CANOE
Coinbase   =  
CoinbaseStr=  b''
Addesses   =  147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq
addr_match =  [('BTC.TOP', {'addr_match': '147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq'}), ('CANOE', {'addr_match': '147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq'})]
cb_match   =  None

Message    =  Multiple addresses match
Blockheight=  522814
Miner1     =  BTC.TOP
Miner2     =  CANOE
Coinbase   =  
CoinbaseStr=  b''
Addesses   =  147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq
addr_match =  [('BTC.TOP', {'addr_match': '147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq'}), ('CANOE', {'addr_match': '147Sw


Message    =  Multiple addresses match
Blockheight=  525885
Miner1     =  BTC.TOP
Miner2     =  CANOE
Coinbase   =  
CoinbaseStr=  b''
Addesses   =  147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq
addr_match =  [('BTC.TOP', {'addr_match': '147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq'}), ('CANOE', {'addr_match': '147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq'})]
cb_match   =  None

Message    =  Multiple addresses match
Blockheight=  525904
Miner1     =  BTC.TOP
Miner2     =  CANOE
Coinbase   =  
CoinbaseStr=  b''
Addesses   =  147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq
addr_match =  [('BTC.TOP', {'addr_match': '147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq'}), ('CANOE', {'addr_match': '147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq'})]
cb_match   =  None
526000
Message    =  Multiple addresses match
Blockheight=  526374
Miner1     =  BTC.TOP
Miner2     =  CANOE
Coinbase   =  
CoinbaseStr=  b''
Addesses   =  147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq
addr_match =  [('BTC.TOP', {'addr_match': '147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq'}), ('CANOE', {'addr_match': 

538000
Message    =  Multiple addresses match
Blockheight=  538356
Miner1     =  BTC.TOP
Miner2     =  CANOE
Coinbase   =  
CoinbaseStr=  b''
Addesses   =  147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq
addr_match =  [('BTC.TOP', {'addr_match': '147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq'}), ('CANOE', {'addr_match': '147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq'})]
cb_match   =  None
539000
Message    =  Multiple addresses match
Blockheight=  539426
Miner1     =  BTC.TOP
Miner2     =  CANOE
Coinbase   =  
CoinbaseStr=  b''
Addesses   =  147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq
addr_match =  [('BTC.TOP', {'addr_match': '147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq'}), ('CANOE', {'addr_match': '147SwRQdpCfj5p8PnfsXV2SsVVpVcz3aPq'})]
cb_match   =  None


In [184]:
len(conflicts) # 479 # 455

455

In [185]:
with open(miners_initial_conflicts_json_file, 'w') as fp:
    json.dump(conflicts, fp)

Redo/Update the blocks json attribution of blocktrail to unify the name of digitalX Mintsy in this
attribution data

In [186]:
# blocktrail.com again to map 'digitalX Mintsy' to 'digitalBTC' correctly

for blknum in blocks:
    # iterate over all blocks once
    if blknum in blocks_blocktrail.keys():
        # check if block was attributed by blocktrail
        
        # !!! remove old blocktrail.com attribution
        blocks[ blknum ][ util.D_ATTRIBUTIONS ].pop( util.DD_BT_ATTR, None)
        
        miner_bt = blocks_blocktrail[ blknum ][ util.DDD_MINER ]
        if miner_bt == "unknown" or miner_bt == "Unknown Entity":
            continue
            
        miner_uid = miner_bt
        mapped = False
        for miner_id in miners:
            # check if there is a unified miner id for this miner already in miners.json
            if miner_bt in miners[ miner_id ][ util.D_NAMES ].keys():
                # unified miner_id found
                miner_uid = miner_id
                mapped = True
                break
                
        if not mapped and len( blocks[ blknum ][ util.D_ADDRESSES ] ) == 1:
            # if miner name could not be mapped to list of existing miner names
            # it is probably a new miner and gets added as such
            address = blocks[ blknum ][ util.D_ADDRESSES ][0]
            print(miner_uid,":",address)
            util.add_miner(miner_uid,
                     miners,
                     names_dict= { miner_uid: { util.DD_URL:"",
                                            util.DD_CURRENCIES: ["BTC",],
                                            util.DD_FULLNAME: "",
                                            util.DD_FIRSTUSED: 0,
                                            util.DD_LASTUSED: 0,
                                            util.DD_SOURCES:["blocktrail.com",] } },
                     addresses_dict= { address: { util.DD_CURRENCIES: ["BTC",],
                                              util.DD_FIRSTUSED: 0,
                                              util.DD_LASTUSED: 0,
                                              util.DD_SOURCES:["blocktrail.com",] } }, )
        elif mapped and len( blocks[ blknum ][ util.D_ADDRESSES ] ) == 1:
            # the the miner could be mapped then check if we already have the address 
            # of the coinbase output mapped to this miner 
            # (if there is only one address in the block coinbase output)
            # If we dont have the address add it, if we have it add "blocktrail.com" as source to the address
            address = blocks[ blknum ][ util.D_ADDRESSES ][0]
            """
            if address == "1FLH1SoLv4U68yUERhDiWzrJn5TggMqkaZ":
                # check for specific address 
                for attr in blocks[ blknum ][ util.D_ATTRIBUTIONS ].keys():
                    if blocks[ blknum ][ util.D_ATTRIBUTIONS ][ attr ][ "miner" ] != miner_uid:
                        print(blknum,":",blocks[ blknum ][ util.D_ATTRIBUTIONS ][ attr ][ "miner" ],"--",miner_uid)
            """            
            util.add_addr(miner_uid,miners,address,source="blocktrail.com",currencies=["BTC",])
        
        # attribute block based on miner_uid in any case
        blocks[ blknum ][ util.D_ATTRIBUTIONS ][ util.DD_BT_ATTR ] = { util.DDD_MINER:miner_uid,
                                                                       util.DDD_SRC:"blocktrail.com" }       
        

Reapply fixes and checks to see if everything is consistant 

In [187]:
# first re-undo the incorrec attribution of blocktrail which messes up BTCC and F2Pool:
print(miners["BTCC Pool"][ util.D_ADDRESSES ][ "1KFHE7w8BhaENAswwryaoccDb6qcT6DbYY" ])
miners["BTCC Pool"][ util.D_ADDRESSES ].pop( "1KFHE7w8BhaENAswwryaoccDb6qcT6DbYY" , None)

{'sources': ['blocktrail.com'], 'firstUsed': 0, 'lastUsed': 0, 'currencies': ['BTC']}


{'sources': ['blocktrail.com'],
 'firstUsed': 0,
 'lastUsed': 0,
 'currencies': ['BTC']}

In [188]:
for m in miners:
    if util.D_ADDRESSES in miners[ m ].keys():
        for addr in miners[ m ][ util.D_ADDRESSES ]:
            if addr == "1KFHE7w8BhaENAswwryaoccDb6qcT6DbYY":
                print(m)
                pprint.pprint(miners[ m ])

F2Pool
{'addresses': {'13NA7X1u18CgGa7RzTDyvuuoLJtLRvXgke': {'currencies': ['BTC'],
                                                      'firstUsed': 0,
                                                      'lastUsed': 0,
                                                      'sources': ['blocktrail.com']},
               '1KFHE7w8BhaENAswwryaoccDb6qcT6DbYY': {'currencies': ['BTC'],
                                                      'firstUsed': 0,
                                                      'lastUsed': 0,
                                                      'sources': ['blockchain.info '
                                                                  'github',
                                                                  'btccom '
                                                                  'github',
                                                                  'blocktrail.com']},
               '1LoveYoURwCeQu6dURqTQ7hrhYXDA4eJyn': {'currencies': ['BTC'],

## Persist files 

In [189]:
with open(miners_json_file, 'w') as fp:
    json.dump(miners, fp)

In [190]:
with open(blocks_attribution_json_file, 'w') as fp:
    json.dump(blocks, fp)